# SB3 を用いた 2 リンクアームの躍度最小化軌道の学習(https://github.com/DLR-RM/stable-baselines3)
### この jupyternotebook は以下のような構造になっています
1. モジュール管理
2. パラメータ管理
3. 環境設定
4. 環境が正しく動くかの確認
5. エピソードごとの報酬などの学習内部指標を定期的に保存するためのコールバック
6. 評価タイミングごとの評価結果とその時点までに学習したエピソード数をCSV に記録するためのコールバック
7. 学習
8. 学習済みデータを用いたシミュレーション
9. ベストモデルを用いたシミュレーション
10. エピソードごとの様々な報酬の推移などの多指標をプロット

## モジュール管理


In [ ]:
# ===== 基本的なライブラリ =====
import os             # ファイルパスやディレクトリの操作（例：ログ保存先の作成など）
import json           # 設定ファイルや学習結果をJSON形式で保存・読み込むために使用
import csv            # 結果やログをCSV形式で出力するために使用
import time           # 実行時間の計測やスリープ処理などに利用
import glob           # 特定パターンのファイル探索（例："*.csv" など）
from typing import Optional, List  # 型ヒントのため（関数の引数・戻り値の明示）
from datetime import datetime      # 実行開始時刻やログのタイムスタンプ記録に利用
import numpy as np    # 数値計算用ライブラリ（ベクトル・行列演算、乱数生成など）
import pandas as pd   # データ処理やCSVの読み書き・集計・可視化に便利

# ===== 可視化関連 =====
import matplotlib.pyplot as plt  # 学習曲線、報酬推移、動作ログなどの可視化に使用

# ===== 強化学習関連（Stable-Baselines3） =====
from stable_baselines3 import PPO, SAC, TD3, DDPG, A2C  
# → 代表的な強化学習アルゴリズム（方策勾配系やアクタークリティック系）をインポート

from stable_baselines3.common.callbacks import EvalCallback, BaseCallback  
# → 学習中に評価や早期終了などを行うためのコールバック機能を利用するため

from stable_baselines3.common.env_checker import check_env  
# → 独自実装したGym環境がSB3に適合しているかを検証するための関数

from collections import deque  # 一定長の履歴（移動平均など）を保持するのに便利

# ===== Gym関連 =====
import gymnasium as gym  # 強化学習環境の作成・管理（GymnasiumはGymの後継ライブラリ）




## パラメータ管理

In [ ]:
# ===== シミュレーションパラメータ =====
DT = 0.01           # シミュレーションの時間刻み (秒)
L = 1.0             # アームの全長 (m)（ここでは正規化された長さとして 1.0）
STEPS_MAX = 200     # 1エピソードあたりの最大ステップ数
THRESHOLD = 0.03    # ゴール判定の許容距離 (m)

# ===== アクションと状態の範囲 =====
# ★すでに「トルク [N·m]」として使っているのでコメントもそれに合わせておく
ACTION_MIN, ACTION_MAX = -20.0, 20.0  # アクション範囲（各関節トルクの下限・上限）[N·m]
THETA_MIN, THETA_MAX = 0.0, 180.0     # 関節角度の下限・上限 [deg]
THETA_INIT = [40.0, 100.0]            # 初期角度 [deg]
GOAL_POS = [0.0, 0.9]                 # ゴール位置 (x, y) [m]

# --- 物理パラメータ（ヒト上肢オーダーの近似値） ---
# 上腕 ~2 kg, 前腕+手 ~1.5 kg 程度のオーダー
M1 = 1.0   # 上腕リンクの質量 [kg]
M2 = 1.0   # 前腕リンクの質量 [kg]

# 関節粘性（ダンピング）。0.05〜0.2 の範囲で試した中から、
# 学習・軌道の安定性が良かった値として 0.1 を採用。
JOINT_DAMPING = 0.1  # [N·m·s/rad]

# --- 報酬関連（終端 + shaping） ---
REWARD_P_V = 15.0         # 終端速度ペナルティ係数
REWARD_J = 1e-2           # 終端躍度係数（累積にかける）
REWARD_JE_LIM = 1.0       # 比率表示用（infoに出すだけ）

# --- 目標時間と時間軸の扱い（ソフト制約） ---
T_TARGET = 0.45           # 目標所要時間 [s]
SIGMA_T = 2.0             # 時間ボーナスの許容幅 [s]
R_GOAL = 50.0             # 成功時の基礎ボーナス（十分大きくする）
TIME_COST = 1e-3          # 毎ステップの時間コスト（小）

# --- shaping（毎ステップの小さなガイド） ---
SHAPING_DIST_COEFF = 10.0   # 距離改善に対するステップ報酬倍率
SHAPING_JERK_COEFF = 1e-7   # 瞬時躍度ペナルティ係数（小さく）

# --- 停滞検出（進捗が無いと早めに打ち切る） ---
STALL_WINDOW_S = 0.30        # 停滞ウィンドウ長 [s]
STALL_WINDOW = max(1, int(STALL_WINDOW_S / DT))  # ステップ単位
MIN_PROGRESS_PER_WINDOW = 1e-4   # この期間の進捗がこれ未満なら停滞と判定（m）

# ===== 学習設定 =====
TOTAL_TIMESTEPS = 4_000_000   # 学習全体での総ステップ数
LEARNING_RATE = 0.0003        # 学習率
TAU = 0.01                    # SACを使う際のαの学習率（今はPPOなら未使用でもOK）
HID_LAY = 64                  # 隠れ層のノード数
BUFFER_SIZE = 100000          # リプレイバッファのサイズ
BATCH_SIZE = 256              # ミニバッチサイズ
EVAL_FREQ = 100               # 何ステップごとに評価するか
SAVE_INTERVAL = 1             # 何エピソードごとにモデルを保存するか
RUNNNIG_WINDOW = 10           # 直近何エピソードの平均を計算するか
STATES_ARM = 4                # armに関する状態数
SAME_ACTION_TOL = 1e-8        # 同一アクション判定の許容誤差

SUCCESS_WINDOW_SIZE = 100             # sigma_T更新用のウィンドウサイズ
SIGMA_UPDATE_INTERVAL = 10            # sigma_T更新のインターバル
SIGMA_SHARPEN_THRESHOLD = 0.80        # sigmaを小さくする成功率しきい値
SIGMA_BLUNT_THRESHOLD = 0.40          # sigmaを大きくする成功率しきい値
SIGMA_SHARPEN_FACTOR = 0.90           # sigma先鋭化倍率 (<1)
SIGMA_BLUNT_FACTOR = 1.10             # sigma鈍化倍率 (>1)
SIGMA_MIN = 0.2                       # sigmaの下限
SIGMA_MAX_FACTOR = 10.0               # sigmaの上限計算に使う倍率

# ===== stage2（shaping フェーズアウト）関連パラメータ =====
# 成功率と sigma_T が十分に良くなったら stage2 に入り、
# shaping をエピソード単位でフェードアウトさせるためのしきい値
STAGE2_SUCCESS_THRESHOLD = 0.80   # 直近成功率がこの値以上
STAGE2_SIGMA_THRESHOLD   = 0.20   # sigma_T がこの値以下
STAGE2_FADE_EPISODES     = 4000   # stage2 移行後、このエピソード数かけて線形にshapingを 1→0 へ


## 環境設定（2リンクアームのリーチング）

In [ ]:
class TwoJointReachingEnv(gym.Env):
    """
    2関節アームのリーチング環境（ダイナミクス込み + 2段階報酬版）

    - stage1:
        距離shaping, jerk shaping, 時間コスト を使って学習をガイド
    - stage2:
        直近 success-rate と sigma_T を見て条件を満たしたら遷移
        → shaping係数をエピソード単位で徐々に 0 までフェードアウト
        → 終端の jerk / velocity / time bonus が主役の目的関数に移行
    """

    metadata = {"render.modes": ["human"]}

    def __init__(self):
        super().__init__()

        # ---------- 基本パラメータ ----------
        self.dt = DT
        self.l1 = float(L) * 0.5
        self.l2 = float(L) * 0.5

        # --- 物理パラメータ（ダイナミクス用） ---
        # ★ globals() 依存をやめて、上で定義した M1, M2, JOINT_DAMPING をそのまま使う
        self.m1 = M1
        self.m2 = M2
        # COM を人の上肢データに近い 0.43L 付近に設定
        self.lc1 = self.l1 * 0.5
        self.lc2 = self.l2 * 0.5
        # 慣性モーメント（細い棒の中央まわり近似）
        self.I1 = (1.0 / 12.0) * self.m1 * (self.l1 ** 2)
        self.I2 = (1.0 / 12.0) * self.m2 * (self.l2 ** 2)
        self.g = 9.81
        self.b = JOINT_DAMPING  # [N·m·s/rad]

        # エピソード最大ステップ、ゴール閾値、ゴール位置
        self.max_steps = STEPS_MAX
        self.goal_threshold = THRESHOLD
        self.goal_pos = np.array(GOAL_POS, dtype=np.float32)

        # --- 報酬 / タスクパラメータ ---
        self.T_target = T_TARGET
        self.sigma_T = SIGMA_T
        self.R_goal = R_GOAL

        # ★shaping の「元の値」を保持しておく（stage2 フェード用）
        self.base_shaping_dist_coeff = SHAPING_DIST_COEFF
        self.base_shaping_jerk_coeff = SHAPING_JERK_COEFF
        self.base_time_cost = TIME_COST

        # 実際に step で使う係数（毎ステップ更新）
        self.shaping_dist_coeff = float(self.base_shaping_dist_coeff)
        self.shaping_jerk_coeff = float(self.base_shaping_jerk_coeff)
        self.time_cost = float(self.base_time_cost)

        # 終端ペナルティの係数
        self.terminal_jerk_coeff = REWARD_J
        self.terminal_vel_coeff = REWARD_P_V

        # 停滞判定
        self.stall_window = STALL_WINDOW
        self.min_progress_per_window = MIN_PROGRESS_PER_WINDOW

        # ---------- 観測空間 ----------
        obs_low = np.array([-1.0] * STATES_ARM + [0.0], dtype=np.float32)
        obs_high = np.array([1.0] * STATES_ARM + [1.0], dtype=np.float32)
        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, dtype=np.float32)

        # ---------- アクション空間（トルク） ----------
        self.torque_min = ACTION_MIN
        self.torque_max = ACTION_MAX
        self.action_space = gym.spaces.Box(
            low=np.array([self.torque_min, self.torque_min], dtype=np.float32),
            high=np.array([self.torque_max, self.torque_max], dtype=np.float32),
            shape=(2,),
            dtype=np.float32
        )
        self._same_action_counter = 0
        self._same_action_tol = SAME_ACTION_TOL

        # ---------- sigma 自動更新 ----------
        self._success_window_size = SUCCESS_WINDOW_SIZE
        self._success_window = deque(maxlen=self._success_window_size)

        self.episode_count = 0
        self.sigma_update_interval = SIGMA_UPDATE_INTERVAL
        self._sigma_update_counter = 0

        self.sigma_sharpen_threshold = SIGMA_SHARPEN_THRESHOLD
        self.sigma_blunt_threshold = SIGMA_BLUNT_THRESHOLD
        self.sigma_sharpen_factor = SIGMA_SHARPEN_FACTOR
        self.sigma_blunt_factor = SIGMA_BLUNT_FACTOR
        self.sigma_min = SIGMA_MIN
        self.sigma_max = max(1e-3, SIGMA_T * SIGMA_MAX_FACTOR)

        self._last_sigma_update = {
            "episode": None,
            "old_sigma": None,
            "new_sigma": None,
            "success_rate": None,
            "action": None,
        }

        # ---------- ★stage2（shaping フェーズアウト）関連 ----------
        # ★ここも globals() の分岐をやめて、上で定義した値をそのまま使う
        self.stage2_success_threshold = STAGE2_SUCCESS_THRESHOLD
        self.stage2_sigma_threshold = STAGE2_SIGMA_THRESHOLD
        self.stage2_fade_episodes = STAGE2_FADE_EPISODES

        self.stage2_active = False          # stage2 に入ったかどうか
        self.stage2_start_episode = None    # stage2 開始エピソード
        self.stage2_last_success_rate = None  # 判定に使った成功率の記録

        # 環境初期化
        self.reset()

    # ---------- ヘルパー ----------
    def set_penalty_weight(self, new_weight):
        self.terminal_jerk_coeff = float(new_weight)

    def reached_goal(self, dist_to_goal):
        return float(dist_to_goal) <= float(self.goal_threshold)

    def forward_kinematics(self, thetas):
        th1, th2 = float(thetas[0]), float(thetas[1])
        x = self.l1 * np.cos(th1) + self.l2 * np.cos(th1 + th2)
        y = self.l1 * np.sin(th1) + self.l2 * np.sin(th1 + th2)
        return np.array([x, y], dtype=np.float32)

    def _get_obs(self):
        pos_scale = (self.l1 + self.l2)
        hand_pos_scaled = np.asarray(self.hand_pos / max(1e-8, pos_scale)).ravel()
        hand_pos_scaled = np.clip(hand_pos_scaled, -1.0, 1.0)

        # （便宜上）トルクレンジから最大角速度スケールを近似
        max_joint_speed_rad = np.radians(max(abs(self.torque_min), abs(self.torque_max)))
        max_hand_speed = max_joint_speed_rad * pos_scale
        hand_vel_scaled = np.asarray(self.hand_vel / max(1e-8, max_hand_speed)).ravel()
        hand_vel_scaled = np.clip(hand_vel_scaled, -1.0, 1.0)

        time_scaled = np.array(
            [np.clip(float(self.steps) / max(1, int(self.max_steps)), 0.0, 1.0)],
            dtype=np.float32
        )

        obs = np.concatenate([
            hand_pos_scaled.astype(np.float32),
            hand_vel_scaled.astype(np.float32),
            time_scaled.astype(np.float32),
        ])
        return obs

    # ---------- Gymnasium API: reset ----------
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)

        self.episode_count += 1

        try:
            th_init_arr = np.asarray(THETA_INIT)
            if th_init_arr.size == 1:
                th_init_arr = np.array([th_init_arr.item(), th_init_arr.item()], dtype=np.float32)
            else:
                th_init_arr = th_init_arr.reshape(-1)[:2].astype(np.float32)
        except Exception:
            th_init_arr = np.array([THETA_INIT, THETA_INIT], dtype=np.float32)

        self.theta = np.radians(th_init_arr).astype(np.float32)
        self.steps = 0
        self.t = 0.0

        self.theta_vel = np.zeros(2, dtype=np.float32)
        self.theta_acc = np.zeros(2, dtype=np.float32)
        self.theta_jerk = np.zeros(2, dtype=np.float32)

        self.hand_pos = self.forward_kinematics(self.theta)
        self.hand_vel = np.zeros(2, dtype=np.float32)
        self.hand_acc = np.zeros(2, dtype=np.float32)
        self.hand_jerk = np.zeros(2, dtype=np.float32)

        self.jerk_sum = 0.0

        self.prev_dist = np.linalg.norm(self.hand_pos - self.goal_pos)
        self.dist_window = [self.prev_dist]

        self._same_action_counter = 0

        # ★reset のたびに、stage2 のフェードアウト係数に応じて shaping 係数を更新
        self._update_shaping_coeffs_for_current_episode()

        return self._get_obs(), {}

    # ---------- ★stage2 用の shaping 更新ロジック ----------
    def _get_stage2_shaping_scale(self) -> float:
        """
        stage2 中であれば、エピソード数に応じて [1 → 0] へ線形に減衰させるスケールを返す。
        stage2 でなければ 1.0 を返す。
        """
        if not self.stage2_active or self.stage2_start_episode is None:
            return 1.0

        elapsed = max(0, self.episode_count - self.stage2_start_episode)
        k = elapsed / max(1, self.stage2_fade_episodes)  # 0〜1 以上
        scale = float(np.clip(1.0 - k, 0.0, 1.0))
        return scale

    def _update_shaping_coeffs_for_current_episode(self):
        """
        現在の episode_count / stage2 状態に応じて
        shaping_dist_coeff / shaping_jerk_coeff / time_cost を更新する。
        """
        scale = self._get_stage2_shaping_scale()
        self.shaping_dist_coeff = float(self.base_shaping_dist_coeff * scale)
        self.shaping_jerk_coeff = float(self.base_shaping_jerk_coeff * scale)
        self.time_cost = float(self.base_time_cost * scale)

    # ---------- 2リンクアームの前進ダイナミクス ----------
    def compute_forward_dynamics(self, theta, theta_vel, tau):
        th1, th2 = float(theta[0]), float(theta[1])
        dth1, dth2 = float(theta_vel[0]), float(theta_vel[1])

        c2 = np.cos(th2)
        s2 = np.sin(th2)

        m1, m2 = self.m1, self.m2
        l1, lc1, lc2 = self.l1, self.lc1, self.lc2
        I1, I2 = self.I1, self.I2

        M11 = I1 + I2 + m1 * (lc1 ** 2) + m2 * (l1 ** 2 + lc2 ** 2 + 2 * l1 * lc2 * c2)
        M22 = I2 + m2 * (lc2 ** 2)
        M12 = I2 + m2 * (lc2 ** 2 + l1 * lc2 * c2)
        M21 = M12
        M = np.array([[M11, M12],
                      [M21, M22]], dtype=np.float32)

        h = - m2 * l1 * lc2 * s2
        C1 = h * dth2 * (2.0 * dth1 + dth2)
        C2 = h * dth1 * dth1
        C_vec = np.array([C1, C2], dtype=np.float32)

        g = self.g
        G1 = (m1 * lc1 + m2 * l1) * g * np.cos(th1) + m2 * lc2 * g * np.cos(th1 + th2)
        G2 = m2 * lc2 * g * np.cos(th1 + th2)
        G_vec = np.array([G1, G2], dtype=np.float32)

        B_vec = self.b * np.asarray(theta_vel, dtype=np.float32)

        rhs = np.asarray(tau, dtype=np.float32) - C_vec - G_vec - B_vec
        theta_acc = np.linalg.solve(M, rhs)
        return theta_acc.astype(np.float32)

    # ---------- Sigma 更新 + ★stage2 判定 ----------
    def update_sigma_by_success_rate(self):
        """
        sigma_T 更新に加えて、
        - success-rate
        - sigma_T
        を見て stage2 開始条件を満たしたら stage2_active を True にする。
        """
        if len(self._success_window) < self._success_window_size:
            return False

        success_rate = float(sum(self._success_window)) / float(self._success_window_size)
        old_sigma = float(self.sigma_T)
        update_action = None

        if success_rate >= self.sigma_sharpen_threshold:
            new_sigma = old_sigma * float(self.sigma_sharpen_factor)
            update_action = "sharpen"
        elif success_rate <= self.sigma_blunt_threshold:
            new_sigma = old_sigma * float(self.sigma_blunt_factor)
            update_action = "blunt"
        else:
            new_sigma = old_sigma

        new_sigma = float(np.clip(new_sigma, self.sigma_min, self.sigma_max))
        self.sigma_T = new_sigma

        self._last_sigma_update = {
            "episode": int(self.episode_count),
            "old_sigma": float(old_sigma),
            "new_sigma": float(new_sigma),
            "success_rate": float(success_rate),
            "action": update_action,
        }

        # ★ここで stage2 開始判定
        self.stage2_last_success_rate = float(success_rate)
        if (not self.stage2_active
            and success_rate >= self.stage2_success_threshold
            and self.sigma_T <= self.stage2_sigma_threshold):
            self.stage2_active = True
            self.stage2_start_episode = int(self.episode_count)
            # 次の reset から shaping 係数がフェードアウトを開始

        return True

    # ---------- Gymnasium API: step ----------
    def step(self, action):
        prev_theta = self.theta.copy()
        prev_vel = self.theta_vel.copy()
        prev_acc = self.theta_acc.copy()

        tau = np.asarray(action, dtype=np.float32).reshape(-1)
        if tau.shape[0] != 2:
            raise ValueError(f"action must have shape (2,), got {tau.shape}")

        tau_clipped = np.clip(tau, self.torque_min, self.torque_max)

        action_same = bool(np.allclose(tau_clipped[0], tau_clipped[1], atol=self._same_action_tol))
        if action_same:
            self._same_action_counter += 1
        else:
            self._same_action_counter = 0

        # --- ダイナミクス更新 ---
        self.theta_acc = self.compute_forward_dynamics(prev_theta, prev_vel, tau_clipped)
        self.theta_vel = prev_vel + self.theta_acc * self.dt
        self.theta = prev_theta + self.theta_vel * self.dt

        theta_min = np.radians(THETA_MIN)
        theta_max = np.radians(THETA_MAX)
        self.theta = np.clip(self.theta, theta_min, theta_max)

        self.theta_jerk = (self.theta_acc - prev_acc) / self.dt

        prev_hand_pos = self.hand_pos.copy()
        prev_hand_vel = self.hand_vel.copy()
        prev_hand_acc = self.hand_acc.copy()

        self.hand_pos = self.forward_kinematics(self.theta)
        self.hand_vel = (self.hand_pos - prev_hand_pos) / self.dt
        self.hand_acc = (self.hand_vel - prev_hand_vel) / self.dt
        self.hand_jerk = (self.hand_acc - prev_hand_acc) / self.dt

        hand_jerk_norm_sq = float(np.dot(self.hand_jerk, self.hand_jerk))
        self.jerk_sum += hand_jerk_norm_sq * self.dt

        self.steps += 1
        self.t += self.dt

        dist_to_goal = float(np.linalg.norm(self.hand_pos - self.goal_pos))

        # ---------- 報酬計算 ----------
        reward_dist_step = self.shaping_dist_coeff * (self.prev_dist - dist_to_goal)
        reward_jerk_step = - self.shaping_jerk_coeff * hand_jerk_norm_sq * self.dt
        reward_time_step = - self.time_cost
        reward = reward_dist_step + reward_jerk_step + reward_time_step

        self.dist_window.append(dist_to_goal)
        if len(self.dist_window) > self.stall_window:
            self.dist_window.pop(0)

        terminated = False
        truncated = False

        terminal_jerk_penalty = 0.0
        terminal_vel_penalty = 0.0
        time_bonus = 0.0

        if self.reached_goal(dist_to_goal):
            time_bonus = self.R_goal * np.exp(
                - (self.t - self.T_target) ** 2 / (2 * max(1e-12, (self.sigma_T ** 2)))
            )
            terminal_jerk_penalty = - self.terminal_jerk_coeff * (self.jerk_sum / max(1, self.steps))
            hand_vel_norm_sq = float(np.dot(self.hand_vel, self.hand_vel))
            terminal_vel_penalty = - self.terminal_vel_coeff * hand_vel_norm_sq

            reward += time_bonus + terminal_jerk_penalty + terminal_vel_penalty
            terminated = True

        elif len(self.dist_window) >= self.stall_window:
            prog = self.dist_window[0] - self.dist_window[-1]
            if prog < self.min_progress_per_window:
                truncated = True
                reward += -10.0

        if not terminated and self.steps >= self.max_steps:
            truncated = True
            reward += - self.terminal_jerk_coeff * (self.jerk_sum / max(1, self.steps))

        reward_components = {
            "reward_dist_step": float(reward_dist_step),
            "reward_jerk_step": float(reward_jerk_step),
            "reward_time_step": float(reward_time_step),
            "terminal_jerk_penalty": float(terminal_jerk_penalty),
            "terminal_vel_penalty": float(terminal_vel_penalty),
            "time_bonus": float(time_bonus),
            "jerk_sum": float(self.jerk_sum),
            "hand_jerk_norm_sq": hand_jerk_norm_sq,
        }

        info = {
            "hand_pos": self.hand_pos,
            "hand_vel": self.hand_vel,
            "hand_acc": self.hand_acc,
            "hand_jerk": self.hand_jerk,
            "theta": self.theta.copy(),
            "theta_vel": self.theta_vel.copy(),
            "theta_acc": self.theta_acc.copy(),
            "theta_jerk": self.theta_jerk.copy(),
            "dist_to_goal": dist_to_goal,
            "t": float(self.t),
            "action": tau_clipped.copy(),
            "action_same": action_same,
            "same_action_counter": int(self._same_action_counter),
            "reward_total": float(reward),
            "sigma_T": float(self.sigma_T),
            "sigma_update": dict(self._last_sigma_update),
            "stage2_active": bool(self.stage2_active),
            "stage2_start_episode": self.stage2_start_episode,
            "stage2_success_rate": self.stage2_last_success_rate,
            "shaping_scale": self._get_stage2_shaping_scale(),
            **reward_components,
        }

        # --- 終端時の success-window & sigma 更新 ---
        if terminated or truncated:
            success_flag = 1 if terminated else 0
            self._success_window.append(success_flag)

            if len(self._success_window) == self._success_window_size:
                self._sigma_update_counter += 1
                if (self._sigma_update_counter % self.sigma_update_interval) == 0:
                    _ = self.update_sigma_by_success_rate()
                    # sigma / success_rate が変わったら次エピソードから shaping 更新される

        self.prev_dist = dist_to_goal
        return self._get_obs(), float(reward), bool(terminated), bool(truncated), info

    # ---------- 描画 ----------
    def render(self):
        print(
            f"t={self.t:.3f}s step={self.steps}, "
            f"theta1={np.degrees(self.theta[0]):.2f} deg, "
            f"theta2={np.degrees(self.theta[1]):.2f} deg, "
            f"hand={self.hand_pos}, sigma_T={self.sigma_T:.6f}, "
            f"stage2={self.stage2_active}"
        )


## 環境が動くかの確認

In [ ]:
env = TwoJointReachingEnv()
check_env(env, warn=True)

## エピソードごとの報酬などの学習内部指標を定期的に保存するためのコールバック

In [ ]:
"""
FullMonitorCallback
====================

役割:
  - Stable-Baselines3（PPO/SAC など）での学習中に、
    「エピソード単位の指標」を CSV に記録するためのコールバック。

設計方針:
  - 環境の `info` に入ってくる「報酬の内訳」や `sigma_T` など、
    アルゴリズム非依存の情報だけを集約する。
  - 1 エピソードぶんの情報をバッファに貯めておき、
    エピソード終了時に 1 行追記することで、後から解析しやすいログを作る。

関連コンポーネント:
  - TwoJointReachingEnv: `info` に報酬内訳 / sigma_T / shaping_scale などを詰めている環境クラス。
  - EvalWithTrainEpisodeCallback: 評価タイミングと学習エピソード数を紐づける EvalCallback 拡張。
"""

class FullMonitorCallback(BaseCallback):
    """
    エピソードごとの指標を CSV に保存するコールバック（SB3 用）。

    記録する列:
      - episode                    : エピソード番号（1, 2, 3, ...）
      - episode_length             : そのエピソードのステップ数
      - total_reward               : エピソード内報酬の合計
      - sum_reward_dist_step       : 距離 shaping 報酬のステップ和
      - sum_reward_jerk_step       : 躍度 shaping ペナルティのステップ和
      - sum_reward_time_step       : 毎ステップの時間コストのステップ和
      - sum_terminal_jerk_penalty  : 終端躍度ペナルティの合計
      - sum_terminal_vel_penalty   : 終端速度ペナルティの合計
      - sum_time_bonus             : 時間ボーナスの合計
      - jerk_sum                   : エピソード全体の躍度積分（環境側で計算）
      - running_mean_total_reward_N: 直近 N エピソードの total_reward の移動平均
      - episode_wall_time          : 1 エピソードにかかった実時間（wall clock）
      - sigma_T                    : エピソード終端時点の sigma_T
      - shaping_scale              : stage2 フェーズアウト用スケール（0〜1）
    """

    # ------------------------------------------------------------
    # コンストラクタ: ログ先ディレクトリ / 保存間隔 / 窓幅などを受け取る
    # ------------------------------------------------------------
    def __init__(
        self,
        log_dir: str,
        save_interval: int = 1,
        running_window: int = 10,
        verbose: int = 0,
    ):
        """
        Parameters
        ----------
        log_dir : str
            CSV を出力するルートディレクトリ（run_id ごとなど）。
        save_interval : int
            何エピソードごとに 1 行書き込むか。
            - 1 の場合: 毎エピソード追記（デバッグ向き）
            - >1 の場合: 間引いて書き込む（大規模学習向き）
        running_window : int
            直近何エピソードで「平均 total_reward」を取るか（移動平均）。
        verbose : int
            1 の場合、保存タイミングなどを print する。
        """
        super().__init__(verbose)

        # --- ログディレクトリを確実に作成 ---
        self.log_dir = log_dir
        os.makedirs(self.log_dir, exist_ok=True)

        # --- 保存間隔 / 移動平均窓幅 ---
        self.save_interval = int(save_interval)
        self.running_window = int(running_window)

        # --- 出力 CSV のパス ---
        self.csv_path = os.path.join(self.log_dir, "episode_full_metrics.csv")

        # --------------------------------------------------------
        # CSV ヘッダ定義
        #   解析で使う列だけに絞ることで、ログの見通しを良くしている。
        # --------------------------------------------------------
        header = [
            "episode",
            "episode_length",
            "total_reward",
            "sum_reward_dist_step",
            "sum_reward_jerk_step",
            "sum_reward_time_step",
            "sum_terminal_jerk_penalty",
            "sum_terminal_vel_penalty",
            "sum_time_bonus",
            "jerk_sum",
            f"running_mean_total_reward_{self.running_window}",
            "episode_wall_time",
            "sigma_T",
            "shaping_scale",
        ]

        # run ごとにヘッダを書き込む（以降は追記モードで行を追加）
        with open(self.csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(header)

        # --------------------------------------------------------
        # エピソード内の一時バッファ
        #   SB3 の callback は step 単位で呼ばれるので、
        #   1 エピソードぶんをここに貯めてから終端で 1 行にまとめる。
        # --------------------------------------------------------
        self._step_rewards = []   # 各ステップの報酬
        self._step_infos = []     # 各ステップの info（shallow copy）

        # エピソードカウンタ（1, 2, 3, ...）
        self.episode_count = 0

        # 直近 N エピソード分の集計用（移動平均 total_reward 用）
        self.episode_records = []

        # エピソード開始時刻（wall clock）
        self._ep_start_time = None

        # 学習全体の開始時刻（必要に応じて time/系を増やせるよう保持）
        self._train_start_time = None

    # ------------------------------------------------------------
    # helper: 学習開始時に呼ばれる
    # ------------------------------------------------------------
    def _on_training_start(self) -> None:
        """
        学習開始時に呼ばれるフック。
        ここでは「学習開始時刻（wall clock）」を記録する。
        """
        self._train_start_time = time.time()

    # ------------------------------------------------------------
    # helper: info 内の特定キーの値をエピソード内で合計する
    # ------------------------------------------------------------
    def _sum_info_key(self, key: str) -> float:
        """
        環境側の info に入っている `key` の値を、
        1 エピソード分すべてのステップで合計して返す。

        例:
          key="reward_dist_step" → shaping 距離報酬のステップ和。
        """
        return float(sum([it.get(key, 0.0) for it in self._step_infos]))

    # ------------------------------------------------------------
    # main: 各 step ごとに呼ばれる
    #   - ここでは「バッファリング」と「エピソード終了検出」を行う。
    # ------------------------------------------------------------
    def _on_step(self) -> bool:
        # --------------------------------------------------------
        # 1. SB3 の locals から reward / info / done を取り出す
        #    VecEnv 前提で list/ndarray になるため 0 番だけを見る（1 env 想定）。
        # --------------------------------------------------------
        infos = self.locals.get("infos")
        rewards = self.locals.get("rewards")

        # info の取り出し
        if isinstance(infos, (list, tuple, np.ndarray)):
            info = infos[0] if len(infos) > 0 else {}
        else:
            info = infos or {}

        # reward の取り出し
        if isinstance(rewards, (list, tuple, np.ndarray)):
            reward = float(rewards[0])
        else:
            reward = float(rewards or 0.0)

        # ステップごとの値をバッファに積む
        self._step_rewards.append(reward)
        self._step_infos.append(dict(info))  # shallow copy にしておく

        # done 判定の取得（1 env 前提で 0 番だけ見る）
        dones = self.locals.get("dones")
        if isinstance(dones, (list, tuple, np.ndarray)):
            done_flag = bool(dones[0])
        else:
            done_flag = bool(dones)

        # エピソード開始時刻の初期化
        if self._ep_start_time is None:
            self._ep_start_time = time.time()

        # --------------------------------------------------------
        # 2. エピソード終了時の処理
        #    - エピソード内バッファを集約して 1 行の CSV レコードを作る。
        # --------------------------------------------------------
        if done_flag:
            # ---- 基本情報（エピソード番号 / 長さ / 総報酬）----
            self.episode_count += 1
            ep_len = len(self._step_rewards)
            total_reward = float(np.sum(self._step_rewards)) if self._step_rewards else 0.0

            # VecEnv で wrap されている TwoJointReachingEnv を取得（envs[0] を想定）
            base_env = None
            try:
                base_env = getattr(self.training_env, "envs", [None])[0]
            except Exception:
                base_env = None

            # ---- 報酬内訳の集約 ----
            sum_reward_dist_step = self._sum_info_key("reward_dist_step")
            sum_reward_jerk_step = self._sum_info_key("reward_jerk_step")
            sum_reward_time_step = self._sum_info_key("reward_time_step")
            sum_terminal_jerk_penalty = self._sum_info_key("terminal_jerk_penalty")
            sum_terminal_vel_penalty = self._sum_info_key("terminal_vel_penalty")
            sum_time_bonus = self._sum_info_key("time_bonus")

            # jerk_sum（基本的には終端 info に入っている想定）
            if self._step_infos and "jerk_sum" in self._step_infos[-1]:
                jerk_sum = float(self._step_infos[-1].get("jerk_sum", 0.0))
            else:
                jerk_sum = float("nan")

            # 成功フラグ:
            #  - 今後成功率を集計したくなったときのために episode_records 内には保持しておく。
            success = bool(self._step_infos[-1].get("success", False)) if self._step_infos else False
            if not success:
                last_dist = float(self._step_infos[-1].get("dist_to_goal", np.inf)) if self._step_infos else np.inf
                goal_thresh = None
                if base_env is not None:
                    goal_thresh = getattr(base_env, "goal_threshold", None)
                    if goal_thresh is None:
                        goal_thresh = getattr(base_env, "THRESHOLD", None)
                if goal_thresh is not None:
                    success = last_dist <= float(goal_thresh)

            # ---- エピソードにかかった実時間（wall clock）----
            ep_wall_time = time.time() - (self._ep_start_time or time.time())
            self._ep_start_time = None

            # ---- 直近 N エピソード分の total_reward を使った移動平均 ----
            self.episode_records.append({
                "episode": self.episode_count,
                "episode_length": ep_len,
                "total_reward": total_reward,
                "success": int(success),
                "jerk_sum": jerk_sum,
            })
            last_N = self.episode_records[-self.running_window:]
            if last_N:
                running_mean_total_reward = float(
                    np.mean([r["total_reward"] for r in last_N])
                )
            else:
                running_mean_total_reward = 0.0

            # ---- sigma_T, shaping_scale の取得 ----
            #      いつ sigma_T や shaping_scale がどう変化していたかを後から追えるように、
            #      エピソード終端時点の値を記録する。
            if self._step_infos:
                last_info = self._step_infos[-1]
                sigma_T_val = float(last_info.get("sigma_T", float("nan")))
                shaping_scale_val = float(last_info.get("shaping_scale", float("nan")))
            else:
                sigma_T_val = float("nan")
                shaping_scale_val = float("nan")

            # ----------------------------------------------------
            # 3. CSV 行の構築 & 書き込み
            # ----------------------------------------------------
            row = [
                self.episode_count,
                ep_len,
                total_reward,
                sum_reward_dist_step,
                sum_reward_jerk_step,
                sum_reward_time_step,
                sum_terminal_jerk_penalty,
                sum_terminal_vel_penalty,
                sum_time_bonus,
                jerk_sum,
                running_mean_total_reward,
                ep_wall_time,
                sigma_T_val,
                shaping_scale_val,
            ]

            # save_interval ごとに追記（毎エピソード書きたい場合は save_interval=1）
            if (self.episode_count % self.save_interval) == 0:
                with open(self.csv_path, "a", newline="") as f:
                    writer = csv.writer(f)
                    writer.writerow(row)
                if self.verbose:
                    print(f"[FullMonitor] Saved episode {self.episode_count} to {self.csv_path}")

            # ----------------------------------------------------
            # 4. SB3 logger にも簡単な指標を流す
            #    - success / running_success_rate は記録しない。
            # ----------------------------------------------------
            try:
                self.logger.record("episode/total_reward", total_reward)
                self.logger.record("episode/jerk_sum", jerk_sum)
                self.logger.record("episode/length", ep_len)
                self.logger.record("episode/running_mean_reward", running_mean_total_reward)
                self.logger.record("episode/sigma_T", sigma_T_val)
                self.logger.record("episode/shaping_scale", shaping_scale_val)
            except Exception:
                # logger が存在しない状況でも学習は続けたいので握りつぶす
                pass

            # ----------------------------------------------------
            # 5. 次エピソードに備えてバッファをクリア
            # ----------------------------------------------------
            self._step_rewards.clear()
            self._step_infos.clear()

        # True を返すことで学習継続（False だと学習停止）
        return True


## 評価タイミングごとの評価結果とその時点までに学習したエピソード数をCSV に記録するためのコールバック


In [ ]:
"""
EvalWithTrainEpisodeCallback
============================

役割:
  - Stable-Baselines3 の EvalCallback を拡張し、
    「評価タイミングごとの評価結果」と「その時点までに学習したエピソード数」を
    CSV に記録するためのコールバック。

設計方針:
  - 評価ロジック自体は EvalCallback（親クラス）に任せる。
    → 本クラスでは「評価が1回終わったタイミングを検知して、ログを整形して残す」
      ことだけに責務を絞る。
  - 学習の進行状況と評価結果をあとからマッチングしやすいように、
    「評価インデックス」「総ステップ数」「mean_reward」「平均エピソード長」
    「評価時点での学習エピソード数」「その時点までのベスト平均報酬」を CSV に残す。
  - どのエピソードでベストが出たかの詳細（best_eval_index, best_train_episode_at_eval）は
    外部で集計可能なため、クラス内では保持しない設計とする。

関連コンポーネント:
  - FullMonitorCallback:
      episode_count を持っており、「学習済みエピソード数」を知るために利用する。
  - TwoJointReachingEnv:
      評価対象となる環境（ここでは env の中身には依存しない）。
"""


class EvalWithTrainEpisodeCallback(EvalCallback):
    """
    EvalCallback を拡張して、
    - 各評価の結果（mean_reward / mean_ep_length）
    - 評価実行時点で何エピソード学習していたか（FullMonitorCallback.episode_count）
    - その時点までのベスト mean_reward（best_mean_reward）
    を CSV に記録するためのクラス。
    """

    # ------------------------------------------------------------
    # コンストラクタ: 評価環境 / 監視用コールバック / CSV パスなどを受け取る
    # ------------------------------------------------------------
    def __init__(
        self,
        eval_env,
        full_monitor_callback=None,
        csv_path=None,
        **kwargs,
    ):
        """
        Parameters
        ----------
        eval_env :
            評価に用いる環境（EvalCallback にそのまま渡す）。
        full_monitor_callback : FullMonitorCallback | None
            学習エピソード数を知るために参照するコールバック。
            - None の場合は train_episode_at_eval が None になる。
        csv_path : str | None
            評価ログを書き出す CSV ファイルパス。
            - None の場合はファイルには書き出さず、属性のみ更新。
        kwargs :
            EvalCallback に引き継ぐ追加の引数（n_eval_episodes, eval_freq など）。
        """
        # 親クラス（EvalCallback）の初期化を先に呼ぶ
        super().__init__(eval_env=eval_env, **kwargs)

        # 学習側のエピソード数を参照するためのコールバック
        self.full_monitor_callback = full_monitor_callback

        # 評価結果を記録する CSV ファイルパス
        self.csv_path = csv_path

        # すでにログ済みの「評価回数」を記録しておく
        # （EvalCallback 側は evaluations_results を蓄積していくので、
        #   それが増えたかどうかで「新しい評価が終わった」ことを検知する）
        self._n_evals_logged = 0

        # これまでに観測した mean_reward の最大値（学習全体のベスト評価値）
        self.best_mean_reward = -np.inf

        # --------------------------------------------------------
        # CSV ファイルのヘッダ行を書き込む（csv_path が指定されている場合）
        # --------------------------------------------------------
        if self.csv_path is not None:
            with open(self.csv_path, "w", newline="") as f:
                writer = csv.writer(f, delimiter=",")
                writer.writerow(
                    [
                        "eval_index",              # 1, 2, 3, ... 評価何回目か
                        "timesteps",               # 評価実行時点の総学習ステップ数
                        "mean_reward",             # 評価エピソード群の平均報酬
                        "mean_ep_length",          # 評価エピソード群の平均エピソード長
                        "train_episode_at_eval",   # 評価時点までに学習したエピソード数
                        "best_mean_reward_so_far", # その時点までのベスト平均報酬
                    ]
                )

    # ------------------------------------------------------------
    # main: 学習ステップごとに呼ばれる
    #   - 親クラスの処理で「必要なタイミングで評価」が走る。
    #   - ここでは「評価が新たに1回終わったか？」を検知して CSV に追記する。
    # ------------------------------------------------------------
    def _on_step(self) -> bool:
        # まず EvalCallback 側の標準処理を実行
        # （指定された eval_freq ごとに評価が回り、evaluations_results 等が更新される）
        continue_training = super()._on_step()

        # --------------------------------------------------------
        # 1. 新しい評価が終わったかどうかをチェック
        #    - evaluations_results は shape: (n_evals, n_eval_episodes)
        #    - 要素数が増えていれば「評価1回分が増えた」とみなせる。
        # --------------------------------------------------------
        current_n_evals = len(self.evaluations_results)
        if current_n_evals > self._n_evals_logged:
            # 今回の評価インデックス（1,2,3,...）
            eval_index = current_n_evals

            # EvalCallback が最後に計算した mean_reward
            mean_reward = float(self.last_mean_reward)

            # ----------------------------------------------------
            # 2. 評価エピソード群の平均エピソード長を求める
            #    - evaluations_length[-1] は「今回の評価で走らせた各エピソード長のリスト」
            # ----------------------------------------------------
            if len(self.evaluations_length) > 0:
                mean_ep_length = float(np.mean(self.evaluations_length[-1]))
            else:
                mean_ep_length = float("nan")

            # ----------------------------------------------------
            # 3. 評価実行時点で何エピソード学習していたかを取得
            #    - FullMonitorCallback が episode_count を持っている想定。
            #    - 無ければ None のまま（学習エピソード数を追跡しないモード）。
            # ----------------------------------------------------
            train_episode_at_eval = None
            if self.full_monitor_callback is not None:
                train_episode_at_eval = self.full_monitor_callback.episode_count

            # ----------------------------------------------------
            # 4. これまでのベスト mean_reward を更新
            #    - 「評価タイミングごとのベスト値」を追うことで、
            #      ログの中から最大値を探さなくても済むようにしている。
            # ----------------------------------------------------
            if mean_reward > self.best_mean_reward:
                self.best_mean_reward = mean_reward

            # ----------------------------------------------------
            # 5. CSV に 1 評価分のレコードを追記
            # ----------------------------------------------------
            if self.csv_path is not None:
                with open(self.csv_path, "a", newline="") as f:
                    writer = csv.writer(f, delimiter=",")
                    writer.writerow(
                        [
                            eval_index,
                            self.num_timesteps,       # 学習済みの総ステップ数
                            mean_reward,
                            mean_ep_length,
                            train_episode_at_eval,
                            self.best_mean_reward,
                        ]
                    )

            # この時点までの評価回数を更新し、
            # 次の _on_step 呼び出しでは「新しい評価が終わっていない」と判断されるようにする。
            self._n_evals_logged = current_n_evals

        # True を返すことで学習継続（EvalCallback の判定も尊重）
        return continue_training


## 学習コード
### PPO モデルと EvalCallback の設定

ここでは、**PPO (Proximal Policy Optimization)** を用いて  
2 関節アーム環境 `TwoJointReachingEnv` の到達動作を学習します。

本研究では、  
- **毎ステップでの躍度（jerk）ペナルティ**
- **距離改善による shaping**
- **目標時間 T に基づく時間ボーナス**
- **stage1 → stage2 での shaping フェードアウト**
を組み合わせた **人らしい運動軌道の獲得**を目的としています。

PPO に渡している主な引数の意味は以下の通りです：

- **total_timesteps**  
  学習に使う総ステップ数（1 ステップ＝1 回の環境遷移）
- **eval_freq**  
  何ステップごとに評価するか（例：2000 → 2000 ステップごと）
- **best_model_save_path**  
  評価結果が最良のモデルを保存するディレクトリ
- **deterministic=True**  
  評価時は確率ではなく決定的に動作させ、ポリシーの純粋な性能を確認
- **render=False**  
  評価時の描画（速度低下するため通常は False）

> 本研究では躍度（jerk）と時間・距離の複数コストを含むため、  
> 学習過程そのものがどのように軌道を変化させるかを EvalCallback で追跡している。

---

## 学習ログの見方

PPO 学習時は、`FullMonitorCallback` によりエピソード単位の詳細な指標が CSV に記録されます。  
そのうち、現在重要なものは以下です：

```plaintext
| episode/                |
|    jerk_sum             | １エピソードの手先躍度の総和（滑らかさの指標）
|    length               | エピソードの総ステップ数
|    running_mean_reward  | 直近 N エピソードの平均総報酬
|    shaping_scale        | shaping が 1→0 に減衰する割合（stage2）
|    sigma_T              | 時間ボーナスの幅（時間戦略の変化指標）
|    total_reward         | １エピソードの総報酬
|
| eval/                   |
|    mean_ep_length       | 評価時の平均ステップ数
|    mean_reward          | 評価時の平均総報酬
|
| time/                   |
|    total_timesteps      | 学習で実行した環境ステップの総数


In [ ]:
# ============================================================
# 1. ログ用ディレクトリの作成 & run ID の生成
# ============================================================

# 実行ごとに一意な ID（YYYY_MM_DD_hh_mm_ss）を振る
run_id = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

# ログ保存用のルートディレクトリ（この下に params / モデル / CSV などを置く）
log_dir = f"./logs/{run_id}"
os.makedirs(log_dir, exist_ok=True)

# ============================================================
# 2. すべてのハイパーパラメータを training_params に集約
#    （コード②で定義した値 + 学習関係）
# ============================================================

training_params = {
    # ----- シミュレーション / 環境パラメータ -----
    "DT": DT,                            # 時間刻み [s]
    "L": L,                              # アーム全長 [m]
    "STEPS_MAX": STEPS_MAX,              # 1エピソードの最大ステップ数
    "THRESHOLD": THRESHOLD,              # ゴール判定距離 [m]

    # ----- アクション・状態空間関連 -----
    "ACTION_MIN": ACTION_MIN,            # トルク最小値 [N·m]
    "ACTION_MAX": ACTION_MAX,            # トルク最大値 [N·m]
    "THETA_MIN": THETA_MIN,              # 関節角度の下限 [deg]
    "THETA_MAX": THETA_MAX,              # 関節角度の上限 [deg]
    "THETA_INIT": THETA_INIT,            # 初期関節角度 [deg]（2関節分）
    "GOAL_POS": GOAL_POS,                # ゴール位置 (x, y) [m]

    # ----- 物理パラメータ（質量・ダンピング） -----
    "M1": M1,                            # 上腕リンク質量 [kg]
    "M2": M2,                            # 前腕リンク質量 [kg]
    "JOINT_DAMPING": JOINT_DAMPING,      # 関節粘性 [N·m·s/rad]

    # ----- 終端報酬まわり -----
    "REWARD_P_V": REWARD_P_V,            # 終端速度ペナルティ係数
    "REWARD_J": REWARD_J,                # 終端躍度ペナルティ係数
    "REWARD_JE_LIM": REWARD_JE_LIM,      # 参考用スケール（info に出すだけ）

    # ----- 目標時間 / 時間ボーナス -----
    "T_TARGET": T_TARGET,                # 目標到達時間 [s]
    "SIGMA_T": SIGMA_T,                  # 時間ボーナスの幅 [s]
    "R_GOAL": R_GOAL,                    # ゴールボーナスの基礎値
    "TIME_COST": TIME_COST,              # 毎ステップの時間コスト

    # ----- shaping 報酬（ステップごとのガイド） -----
    "SHAPING_DIST_COEFF": SHAPING_DIST_COEFF,  # 距離改善の係数
    "SHAPING_JERK_COEFF": SHAPING_JERK_COEFF,  # 瞬時躍度ペナルティ係数

    # ----- 停滞検出（早期打ち切り） -----
    "STALL_WINDOW_S": STALL_WINDOW_S,          # 停滞判定窓の長さ [s]
    "STALL_WINDOW": STALL_WINDOW,              # 停滞判定窓（ステップ数）
    "MIN_PROGRESS_PER_WINDOW": MIN_PROGRESS_PER_WINDOW,  # 停滞判定しきい値 [m]

    # ----- 学習関連（PPO / SAC 共通で使えるように） -----
    "TOTAL_TIMESTEPS": TOTAL_TIMESTEPS,  # 全学習ステップ数
    "LEARNING_RATE": LEARNING_RATE,      # 学習率
    "TAU": TAU,                          # （SAC用）ターゲット更新係数 / α更新など
    "HID_LAY": HID_LAY,                  # MLP の隠れ層ユニット数
    "BUFFER_SIZE": BUFFER_SIZE,          # （SAC用）リプレイバッファサイズ
    "BATCH_SIZE": BATCH_SIZE,            # ミニバッチサイズ
    "EVAL_FREQ": EVAL_FREQ,              # 何ステップごとに評価するか
    "SAVE_INTERVAL": SAVE_INTERVAL,      # 何エピソードごとに保存 or ログを切るか
    "RUNNNIG_WINDOW": RUNNNIG_WINDOW,    # 直近エピソード何本で移動平均を取るか
    "STATES_ARM": STATES_ARM,            # アーム状態次元（hand pos/vel + time など）
    "SAME_ACTION_TOL": SAME_ACTION_TOL,  # 同一アクション判定の許容差

    # ----- sigma_T 自動調整まわり -----
    "SUCCESS_WINDOW_SIZE": SUCCESS_WINDOW_SIZE,      # 成功率を見る窓サイズ
    "SIGMA_UPDATE_INTERVAL": SIGMA_UPDATE_INTERVAL,  # 何回ごとに sigma を更新するか
    "SIGMA_SHARPEN_THRESHOLD": SIGMA_SHARPEN_THRESHOLD,  # σ縮小の成功率しきい値
    "SIGMA_BLUNT_THRESHOLD": SIGMA_BLUNT_THRESHOLD,      # σ拡大の成功率しきい値
    "SIGMA_SHARPEN_FACTOR": SIGMA_SHARPEN_FACTOR,        # σ縮小倍率 (<1)
    "SIGMA_BLUNT_FACTOR": SIGMA_BLUNT_FACTOR,            # σ拡大倍率 (>1)
    "SIGMA_MIN": SIGMA_MIN,                        # σの下限
    "SIGMA_MAX_FACTOR": SIGMA_MAX_FACTOR,          # σ上限の倍率（SIGMA_T * factor）

    # ----- stage2（shaping フェードアウト）関連 -----
    "STAGE2_SUCCESS_THRESHOLD": STAGE2_SUCCESS_THRESHOLD,  # stage2 へ入る成功率しきい値
    "STAGE2_SIGMA_THRESHOLD": STAGE2_SIGMA_THRESHOLD,      # stage2 へ入る σ_T しきい値
    "STAGE2_FADE_EPISODES": STAGE2_FADE_EPISODES,          # stage2 で shaping を 1→0 にするエピソード数
}

# params.json として保存しておくと、後で再現性が取りやすい
with open(os.path.join(log_dir, "params.json"), "w") as f:
    json.dump(training_params, f, indent=4)

# ============================================================
# 3. 環境の作成（学習用 env と評価用 eval_env）
# ============================================================

# 学習用の環境（PPO がサンプルを集める先）
env = TwoJointReachingEnv()

# 評価専用の環境（EvalCallback がポリシーをテストする先）
eval_env = TwoJointReachingEnv()

# ============================================================
# 4. コールバックの準備
#    - FullMonitorCallback: エピソードごとの詳細ログ
#    - EvalWithTrainEpisodeCallback: 定期評価 + その時点のエピソード数
# ============================================================

# エピソードごとの報酬内訳・sigma_T などを CSV に落とすコールバック
full_monitor_cb = FullMonitorCallback(
    log_dir=log_dir,
    verbose=1,  # 1: ある程度ログを表示 / 0: 静かに実行
)

# 評価時のメトリクスを保存する CSV のパス
eval_metrics_path = os.path.join(log_dir, "eval_metrics.csv")

# EvalCallback 拡張版：
# - 定期的に eval_env で評価
# - best_model を保存
# - その評価が学習の何エピソード目だったかも一緒に CSV に出す
eval_callback = EvalWithTrainEpisodeCallback(
    eval_env=eval_env,
    full_monitor_callback=full_monitor_cb,   # 学習済みエピソード数を参照するため
    csv_path=eval_metrics_path,             # 評価ログの保存先
    best_model_save_path=log_dir,           # ベストモデルの保存先
    log_path=log_dir,                       # 評価ログ（SB3 標準形式）保存先
    eval_freq=training_params["EVAL_FREQ"], # 何ステップごとに評価するか
    deterministic=True,                     # 評価時は確率ではなく平均行動で実行
    render=False,                           # 評価中に描画しない（基本 False でOK）
)

# ============================================================
# 5. PPO モデルの作成
# ============================================================

# ポリシーネットワークの構造（hidden 層 2段の MLP）
policy_kwargs = dict(
    net_arch=[training_params["HID_LAY"], training_params["HID_LAY"]]
)

# 今回は PPO を使用（SAC に切り替える場合はここを差し替え）
model = PPO(
    policy="MlpPolicy",                          # MLP ベースのポリシー
    env=env,                                     # 学習環境
    verbose=1,                                   # ログ出力レベル（1: 情報, 0: 無し）
    learning_rate=training_params["LEARNING_RATE"],
    batch_size=training_params["BATCH_SIZE"],
    policy_kwargs=policy_kwargs,
    # ここで ent_coef, vf_coef, gamma などを明示的に渡したければ追加する
    # 例:
    # ent_coef=0.0,
    # vf_coef=0.5,
)

# ============================================================
# 6. 学習ループの実行
#    EvalCallback + FullMonitorCallback を併用
# ============================================================

# SB3 は callback を list で渡すと、順番に呼んでくれる
callback = [eval_callback, full_monitor_cb]

model.learn(
    total_timesteps=training_params["TOTAL_TIMESTEPS"],  # 総ステップ数
    callback=callback,                                   # 評価 & ログ用コールバック
)

# ============================================================
# 7. 学習済みモデルの保存 & ベスト評価情報の表示
# ============================================================

# 最終モデルを保存（ベストモデルとは別に、最後の状態を残しておきたい場合）
final_model_path = os.path.join(log_dir, "final_model.zip")
model.save(final_model_path)
print(f"✅ Final model saved: {final_model_path}")

# EvalWithTrainEpisodeCallback が持っている「ベスト評価」の情報を表示
print("📌 Best eval mean_reward:", eval_callback.best_mean_reward)
print("📌 Best eval index (1,2,3,...):", eval_callback.best_eval_index)
print("📌 Best eval was at train episode:", eval_callback.best_train_episode_at_eval)
print("   → その時点までに学習したエピソード数（FullMonitorCallback.episode_count ベース）")


## 学習終了時点のモデルの活用

In [ ]:
# =============================================================================
# 2関節アーム：1エピソード分のロールアウトをログ＋可視化するスクリプト
# =============================================================================
# 役割:
#   - 学習済みモデル model と環境 env を1エピソードだけ実行し、
#     「時系列ログ（CSV）」と「可視化用の画像ファイル」を logs/ 以下に保存する。
#
# 設計方針:
#   - 研究ノートや論文用にそのまま使えるよう、
#       (1) 解析用: CSV (time, hand pos, joint, torque, norm など)
#       (2) 図用: PNG (軌道＋アーム姿勢、関節動的変数、手先ノルム、トルク)
#     の2種類を必ず出力する。
#   - 可能な限り env / info から「真値」を取りに行き、フォールバックも用意しておく。
#   - 角度は可読性を優先して deg で保存し、内部計算のみ rad を使う。
#   - 既存のノートブックに貼り付けて動くよう、外から env / model / THETA_INIT / L が
#     与えられている前提で、スクリプト部分だけを整理する。
#
# 関連コンポーネント:
#   - env:   TwoJointReachingEnv など「2関節アームの環境」
#   - model: Stable-Baselines3 の学習済みエージェント
#   - ログ:  ./logs 配下に作られる各エピソードのログフォルダ（別箇所で作成済み）
# =============================================================================


# -----------------------------------------------------------------------------
# 1. ログ出力先フォルダ（最新ログフォルダ）を決定
#    → すでに別のコードで env / model の実験を回して logs/XXX ができている前提。
#    → もっとも更新日時が新しいフォルダを「今回の結果の保存先」とみなす。
# -----------------------------------------------------------------------------
logs_dir = "./logs"
log_folders = [
    os.path.join(logs_dir, d)
    for d in os.listdir(logs_dir)
    if os.path.isdir(os.path.join(logs_dir, d))
]
latest_log_folder = max(log_folders, key=os.path.getmtime)
print(f"[INFO] 最新ログフォルダを使用: {latest_log_folder}")

# -----------------------------------------------------------------------------
# 2. ログ用のリストを初期化
#    - DataFrame 化しやすいように「列ごとのリスト」で保持する。
#    - 2関節アーム＋手先ノルム＋トルク＋行動パターン(action_same) を記録。
# -----------------------------------------------------------------------------
# 時刻と手先位置
time_log = []
hand_x, hand_y = [], []

# 関節ごとの角度・角速度・角加速度・角躍度 [deg, deg/s, deg/s^2, deg/s^3]
theta1_log, theta2_log = [], []
theta1_vel_log, theta2_vel_log = [], []
theta1_acc_log, theta2_acc_log = [], []
theta1_jerk_log, theta2_jerk_log = [], []

# トルクと「2関節がほぼ同じトルクかどうか」のフラグ
torque1_log, torque2_log = [], []  # 各ステップの関節トルク [N·m]
action_same_log = []               # 2 関節トルクがほぼ同じかどうか（チェック用）

# 手先の速度・加速度・躍度のノルム ||v||, ||a||, ||jerk||
hand_speed_log = []       # ||v||
hand_acc_norm_log = []    # ||a||
hand_jerk_norm_log = []   # ||jerk||

# -----------------------------------------------------------------------------
# 3. 初期観測を取得し、時刻 t=0 の状態をログに残す
#    - 解析時に「0秒時点」を含めたいので、reset()直後の値も保存しておく。
#    - env / info の仕様変更に耐えられるよう、try/except でフォールバックを用意。
# -----------------------------------------------------------------------------
obs, _ = env.reset()

try:
    # env 側で各種状態が用意されている場合はこちらを優先して使う
    init_theta      = np.asarray(env.theta).reshape(-1)
    init_theta_vel  = np.asarray(env.theta_vel).reshape(-1)
    init_theta_acc  = np.asarray(env.theta_acc).reshape(-1)
    init_theta_jerk = np.asarray(env.theta_jerk).reshape(-1)

    # 手先位置・速度・加速度・躍度も、あれば env から取得
    init_hand_pos  = np.asarray(getattr(env, "hand_pos", env.forward_kinematics(init_theta)))
    init_hand_vel  = np.asarray(getattr(env, "hand_vel", np.zeros(2)))
    init_hand_acc  = np.asarray(getattr(env, "hand_acc", np.zeros(2)))
    init_hand_jerk = np.asarray(getattr(env, "hand_jerk", np.zeros(2)))
except Exception:
    # env に上記属性が存在しない or 形が崩れている場合のフォールバック
    # → 最低限、初期角度 THETA_INIT とゼロ速度から始める。
    init_theta      = np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)])
    init_theta_vel  = np.zeros(2)
    init_theta_acc  = np.zeros(2)
    init_theta_jerk = np.zeros(2)

    init_hand_pos  = np.asarray(getattr(env, "hand_pos", np.array([0.0, 0.0])))
    init_hand_vel  = np.zeros(2)
    init_hand_acc  = np.zeros(2)
    init_hand_jerk = np.zeros(2)

# --- t = 0 をログに追加（角度系は可読性のため deg に変換して保存） ---
t = 0.0
time_log.append(t)

# 手先位置
hand_x.append(float(init_hand_pos[0]))
hand_y.append(float(init_hand_pos[1]))

# 関節角度など（研究メモに貼りやすいよう deg 単位で保存）
theta1_log.append(np.degrees(init_theta[0]))
theta2_log.append(np.degrees(init_theta[1]))
theta1_vel_log.append(np.degrees(init_theta_vel[0]))
theta2_vel_log.append(np.degrees(init_theta_vel[1]))
theta1_acc_log.append(np.degrees(init_theta_acc[0]))
theta2_acc_log.append(np.degrees(init_theta_acc[1]))
theta1_jerk_log.append(np.degrees(init_theta_jerk[0]))
theta2_jerk_log.append(np.degrees(init_theta_jerk[1]))

# 手先ノルム（速度・加速度・躍度）
hand_speed_log.append(float(np.linalg.norm(init_hand_vel)))
hand_acc_norm_log.append(float(np.linalg.norm(init_hand_acc)))
hand_jerk_norm_log.append(float(np.linalg.norm(init_hand_jerk)))

# t=0 ではまだトルクは発生していないので NaN を入れておく
torque1_log.append(np.nan)
torque2_log.append(np.nan)
action_same_log.append(False)

# -----------------------------------------------------------------------------
# 4. ロールアウト本体ループ
#    - 学習済みモデル model からトルクを出し、env.step(action) で1ステップ進める。
#    - 各ステップごとに、手先・関節・トルク・ノルムをログに追記。
# -----------------------------------------------------------------------------
max_steps_to_run = 200  # 可視化用なので上限ステップ数を決めておく

for _ in range(max_steps_to_run):
    # ---------------------------------------------------------
    # 4-1. 学習済みモデルから行動（ここでは2次元トルク）を取得
    #      - deterministic=True: 評価時なので確定的な方策を使用。
    #      - 形状が (2,) であることを明示的にチェックしておく。
    # ---------------------------------------------------------
    action, _ = model.predict(obs, deterministic=True)
    action = np.asarray(action).reshape(-1)
    if action.shape[0] != 2:
        raise RuntimeError(
            f"model.predict returned action with wrong shape {action.shape}. expected (2,)"
        )

    # 環境を1ステップ進める
    obs, reward, done, truncated, info = env.step(action)

    # ---------------------------------------------------------
    # 4-2. 時刻の更新
    #      - env.dt を持っている場合はそれを使い、なければ 0.01 [s] をデフォルトとする。
    # ---------------------------------------------------------
    t += getattr(env, "dt", 0.01)
    time_log.append(t)

    # ---------------------------------------------------------
    # 4-3. info / env から状態を取り出す
    #      - まず info を優先（vecenv などでステップ毎に記録された「その時点の値」が入る想定）。
    #      - なければ env 属性からフォールバック。
    #      - こうしておくと、ログの定義を変えてもここだけ直せばよい。
    # ---------------------------------------------------------
    if info is not None:
        # 手先
        hand_pos = np.asarray(info.get("hand_pos", getattr(env, "hand_pos", np.array([0.0, 0.0]))))
        hand_vel = np.asarray(info.get("hand_vel", getattr(env, "hand_vel", np.zeros(2))))
        hand_acc = np.asarray(info.get("hand_acc", getattr(env, "hand_acc", np.zeros(2))))
        hand_jerk = np.asarray(info.get("hand_jerk", getattr(env, "hand_jerk", np.zeros(2))))

        # 関節角
        theta      = np.asarray(info.get("theta",      getattr(env, "theta",      np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)]))))
        theta_vel  = np.asarray(info.get("theta_vel",  getattr(env, "theta_vel",  np.zeros(2))))
        theta_acc  = np.asarray(info.get("theta_acc",  getattr(env, "theta_acc",  np.zeros(2))))
        theta_jerk = np.asarray(info.get("theta_jerk", getattr(env, "theta_jerk", np.zeros(2))))

        # トルク（action）も info が持っていればそちらを優先
        action_info = np.asarray(info.get("action", action))

        # action_same: 2関節のトルクがほぼ同じかどうかを bool で保存（解析用の簡単な指標）
        action_same = bool(info.get("action_same", np.allclose(action_info[0], action_info[1], atol=1e-8)))
    else:
        # info に何も入っていない場合のフォールバック
        hand_pos = np.asarray(getattr(env, "hand_pos", np.array([0.0, 0.0])))
        hand_vel = np.asarray(getattr(env, "hand_vel", np.zeros(2)))
        hand_acc = np.asarray(getattr(env, "hand_acc", np.zeros(2)))
        hand_jerk = np.asarray(getattr(env, "hand_jerk", np.zeros(2)))

        theta      = np.asarray(getattr(env, "theta",      np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)])))
        theta_vel  = np.asarray(getattr(env, "theta_vel",  np.zeros(2)))
        theta_acc  = np.asarray(getattr(env, "theta_acc",  np.zeros(2)))
        theta_jerk = np.asarray(getattr(env, "theta_jerk", np.zeros(2)))

        action_info = np.asarray(action)
        action_same = bool(np.allclose(action_info[0], action_info[1], atol=1e-8))

    # ---------------------------------------------------------
    # 4-4. 手先位置をログ（軌道図用）
    # ---------------------------------------------------------
    hand_x.append(float(hand_pos[0]))
    hand_y.append(float(hand_pos[1]))

    # ---------------------------------------------------------
    # 4-5. 関節状態をログ（deg に変換して保存）
    # ---------------------------------------------------------
    theta1_log.append(np.degrees(theta[0]))
    theta2_log.append(np.degrees(theta[1]))
    theta1_vel_log.append(np.degrees(theta_vel[0]))
    theta2_vel_log.append(np.degrees(theta_vel[1]))
    theta1_acc_log.append(np.degrees(theta_acc[0]))
    theta2_acc_log.append(np.degrees(theta_acc[1]))
    theta1_jerk_log.append(np.degrees(theta_jerk[0]))
    theta2_jerk_log.append(np.degrees(theta_jerk[1]))

    # ---------------------------------------------------------
    # 4-6. トルクと「action_same」フラグをログ
    # ---------------------------------------------------------
    torque1_log.append(float(action_info[0]))
    torque2_log.append(float(action_info[1]))
    action_same_log.append(bool(action_same))

    # ---------------------------------------------------------
    # 4-7. 手先速度・加速度・躍度のノルムを計算してログ
    #      - スカラー量に落とすことで、図にしたときの傾向が読み取りやすくなる。
    # ---------------------------------------------------------
    hand_speed     = float(np.linalg.norm(hand_vel))      # ||v||
    hand_acc_norm  = float(np.linalg.norm(hand_acc))      # ||a||
    hand_jerk_norm = float(np.linalg.norm(hand_jerk))     # ||jerk||

    hand_speed_log.append(hand_speed)
    hand_acc_norm_log.append(hand_acc_norm)
    hand_jerk_norm_log.append(hand_jerk_norm)

    # 終端条件（done / truncated）でループ終了
    if bool(done) or bool(truncated):
        break

# -----------------------------------------------------------------------------
# 5. ログを DataFrame にまとめて CSV 保存
#    - ここで列名をきちんと決めておくと、後から解析スクリプトで扱いやすい。
# -----------------------------------------------------------------------------
df = pd.DataFrame({
    "Time": time_log,
    "HandX": hand_x,
    "HandY": hand_y,
    "HandSpeed (m/s)": hand_speed_log,
    "HandAcc (m/s^2)": hand_acc_norm_log,
    "HandJerk (m/s^3)": hand_jerk_norm_log,
    "Theta1 (deg)": theta1_log,
    "Theta2 (deg)": theta2_log,
    "Theta1_vel (deg/s)": theta1_vel_log,
    "Theta2_vel (deg/s)": theta2_vel_log,
    "Theta1_acc (deg/s^2)": theta1_acc_log,
    "Theta2_acc (deg/s^2)": theta2_acc_log,
    "Theta1_jer (deg/s^3)": theta1_jerk_log,
    "Theta2_jer (deg/s^3)": theta2_jerk_log,
    "Torque1 (N·m)": torque1_log,
    "Torque2 (N·m)": torque2_log,
    "Action_same": action_same_log
})

# 保存先フォルダとファイル名のベース
output_folder = latest_log_folder
os.makedirs(output_folder, exist_ok=True)
base_filename = os.path.join(output_folder, "end_result_2joint_with_handnorms")

# CSV 保存
csv_path = f"{base_filename}.csv"
df.to_csv(csv_path, index=False)
print(f"[INFO] CSV 保存: {csv_path}")

# -----------------------------------------------------------------------------
# 6. 可視化①: 手先軌道 + アーム姿勢
#    - 到達軌道とアーム姿勢を重ねる図（論文の典型図）を自動で出力する。
# -----------------------------------------------------------------------------
fig1, ax1 = plt.subplots(figsize=(6, 6))

# 肩は原点 (0,0) に固定
shoulder = np.array([0.0, 0.0])

# 手先軌道（青ライン）
ax1.plot(df["HandX"], df["HandY"], linestyle="-", linewidth=1.5, label="end-effector trajectory")

# start / end のマーカー
ax1.scatter(df["HandX"].iloc[0], df["HandY"].iloc[0], color="red", label="start", zorder=5)
ax1.scatter(df["HandX"].iloc[-1], df["HandY"].iloc[-1], color="green", label="end", zorder=5)

# リンク長（env が l1, l2 を持っていればそれを優先）
try:
    l1 = float(getattr(env, "l1", (L * 0.5)))
    l2 = float(getattr(env, "l2", (L * 0.5)))
except Exception:
    # 万が一 env が壊れていても最低限描画できるようにする
    l1 = L * 0.5
    l2 = L * 0.5

# 関節角を rad に変換
theta1_rad = np.radians(df["Theta1 (deg)"].values)
theta2_rad = np.radians(df["Theta2 (deg)"].values)

# 第1リンク末端（肘）の位置
x1 = l1 * np.cos(theta1_rad)
y1 = l1 * np.sin(theta1_rad)

# 第2リンク末端（手先）の位置（forward kinematics）
x2 = x1 + l2 * np.cos(theta1_rad + theta2_rad)
y2 = y1 + l2 * np.sin(theta1_rad + theta2_rad)

# FK で求めた手先とログされた HandX/Y が一致しているか軽くチェック（環境実装のバグ検出用）
try:
    mismatch = np.nanmax(
        np.abs(
            np.column_stack([df["HandX"].values, df["HandY"].values]) -
            np.column_stack([x2, y2])
        )
    )
    if mismatch > 1e-6:
        print(f"[WARN] forward kinematics and logged hand pos mismatch (max diff={mismatch:.6e})")
except Exception:
    # チェックに失敗しても可視化自体は続行
    pass

# 時系列に沿って複数姿勢を薄いグレーで描画
N_POSES = min(15, len(df))  # 図がごちゃごちゃにならないよう最大姿勢数を制限
indices = np.linspace(0, len(df) - 1, N_POSES).astype(int)

for idx in indices:
    xs = [0.0, x1[idx], x2[idx]]
    ys = [0.0, y1[idx], y2[idx]]
    ax1.plot(xs, ys, linewidth=1, color=(0.5, 0.5, 0.5, 0.25), solid_capstyle='round')
    ax1.scatter(
        [xs[1], xs[2]],
        [ys[1], ys[2]],
        s=40,
        edgecolors='k',
        facecolors=(0.9, 0.9, 0.9, 0.6),
        zorder=4
    )

# 強調: start / end 姿勢（色を変えて太めの線で描画）
# start
i0 = 0
xs0 = [0.0, x1[i0], x2[i0]]
ys0 = [0.0, y1[i0], y2[i0]]
ax1.plot(xs0, ys0, linewidth=2, color='tab:red', solid_capstyle='round', zorder=6)
ax1.scatter([xs0[1], xs0[2]], [ys0[1], ys0[2]], s=80, color='tab:red', edgecolors='k', zorder=7)
ax1.scatter(0.0, 0.0, s=100, color='black', zorder=8)  # 肩

# end
ie = len(df) - 1
xse = [0.0, x1[ie], x2[ie]]
yse = [0.0, y1[ie], y2[ie]]
ax1.plot(xse, yse, linewidth=2, color='tab:green', solid_capstyle='round', zorder=6)
ax1.scatter([xse[1], xse[2]], [yse[1], yse[2]], s=100, color='tab:green', edgecolors='k', zorder=7)

# 軸ラベルなどの装飾
ax1.set_xlabel("X [m]")
ax1.set_ylabel("Y [m]")
ax1.grid(True)
ax1.set_aspect('equal', adjustable='box')

# アーム長に応じて描画範囲に余白を持たせる
reach = l1 + l2
ax1.set_xlim(-reach - 0.1 * reach, reach + 0.1 * reach)
ax1.set_ylim(-0.1 * reach,      reach + 0.6 * reach)  # Y 方向は少し上に余裕を持たせる

plt.tight_layout()
plt.savefig(f"{base_filename}_trajectory_with_arms.png", dpi=300)
plt.show()

# -----------------------------------------------------------------------------
# 7. 可視化②: 関節動的変数（角度・速度・加速度・躍度）
#    - 1つの図に Joint1 / Joint2 を重ねて、対称性や差を確認しやすくする。
# -----------------------------------------------------------------------------
fig2, axes = plt.subplots(4, 1, figsize=(8, 12), sharex=True)

# 角度
axes[0].plot(df["Time"], df["Theta1 (deg)"], label="Theta1")
axes[0].plot(df["Time"], df["Theta2 (deg)"], linestyle="--", label="Theta2")
axes[0].set_ylabel("Theta (deg)")
axes[0].legend()
axes[0].grid()
axes[0].set_ylim(0, 180)  # 2リンクアームの典型的な範囲に合わせておく

# 角速度
axes[1].plot(df["Time"], df["Theta1_vel (deg/s)"], label="Vel1")
axes[1].plot(df["Time"], df["Theta2_vel (deg/s)"], linestyle="--", label="Vel2")
axes[1].set_ylabel("Velocity (deg/s)")
axes[1].legend()
axes[1].grid()

# 角加速度
axes[2].plot(df["Time"], df["Theta1_acc (deg/s^2)"], label="Acc1")
axes[2].plot(df["Time"], df["Theta2_acc (deg/s^2)"], linestyle="--", label="Acc2")
axes[2].set_ylabel("Acceleration (deg/s^2)")
axes[2].legend()
axes[2].grid()

# 角躍度
axes[3].plot(df["Time"], df["Theta1_jer (deg/s^3)"], label="Jerk1")
axes[3].plot(df["Time"], df["Theta2_jer (deg/s^3)"], linestyle="--", label="Jerk2")
axes[3].set_ylabel("Jerk (deg/s^3)")
axes[3].set_xlabel("Time (s)")
axes[3].legend()
axes[3].grid()

plt.tight_layout()
plt.savefig(f"{base_filename}_joint_dynamics.png", dpi=300)
plt.show()

# -----------------------------------------------------------------------------
# 8. 可視化③: 手先ノルム（速度・加速度・躍度）
#    - 手先がどのくらい滑らかに動いているかを俯瞰するための図。
# -----------------------------------------------------------------------------
fig3, ax3 = plt.subplots(3, 1, figsize=(8, 9), sharex=True)

ax3[0].plot(df["Time"], df["HandSpeed (m/s)"], label="Hand Speed ||v||")
ax3[0].set_ylabel("Speed (m/s)")
ax3[0].legend()
ax3[0].grid()

ax3[1].plot(df["Time"], df["HandAcc (m/s^2)"], label="Hand Acc ||a||")
ax3[1].set_ylabel("Acc (m/s^2)")
ax3[1].legend()
ax3[1].grid()

ax3[2].plot(df["Time"], df["HandJerk (m/s^3)"], label="Hand Jerk ||jerk||")
ax3[2].set_ylabel("Jerk (m/s^3)")
ax3[2].set_xlabel("Time (s)")
ax3[2].legend()
ax3[2].grid()

plt.tight_layout()
plt.savefig(f"{base_filename}_hand_norms.png", dpi=300)
plt.show()

# -----------------------------------------------------------------------------
# 9. 可視化④: トルクの推移と action_same の確認
#    - control パターン（左右対称かどうかなど）をざっくり確認するための補助図。
# -----------------------------------------------------------------------------
fig4, ax4 = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

# トルク
ax4[0].plot(df["Time"], df["Torque1 (N·m)"], label="Torque1")
ax4[0].plot(df["Time"], df["Torque2 (N·m)"], linestyle="--", label="Torque2")
ax4[0].set_ylabel("Torque (N·m)")
ax4[0].legend()
ax4[0].grid()

# action_same を 0/1 にして描画（2関節トルクがほぼ同じタイミングがどこか一目で分かる）
ax4[1].plot(df["Time"], df["Action_same"].astype(int), label="Action_same (int)")
ax4[1].set_ylabel("Action_same")
ax4[1].set_xlabel("Time (s)")
ax4[1].grid()

plt.tight_layout()
plt.savefig(f"{base_filename}_torques.png", dpi=300)
plt.show()


## ベストモデルの活用

In [ ]:
# =============================================================================
# best_model ロード ＋ 単一エピソード評価・ログ・可視化スクリプト
# =============================================================================
# 役割:
#   - ./logs 以下の「最新の学習ログフォルダ」から best_model.zip（もしくは最新の .zip）を探す
#   - SB3 のどのアルゴリズムか分からない状態からモデルをロードする
#   - ノートブック側で定義済みの env を使って 1 エピソードだけロールアウト
#   - 2 関節アームの状態・手先ノルム・トルクを CSV に保存し、図を PNG として保存する
#
# 設計方針:
#   - 「最新の実験結果を手早く可視化する」ことを目的に、
#       1) 最新 logs フォルダの自動検出
#       2) best_model.zip の自動検出（なければ最新 .zip）
#       3) Gym / Gymnasium 両対応の reset / step ハンドリング
#       4) env / info の仕様が多少変わっても壊れないフォールバック
#     を意識して実装している。
#   - 角度は人間が読みやすい deg でログし、内部計算のみ rad を用いる。
#   - 「なぜその値を優先するか（info を優先、env をフォールバック）」など、
#     後から読んだときに迷わないようにコメントで方針を明示する。
#
# 関連コンポーネント:
#   - env       : TwoJointReachingEnv 等の 2 関節アーム環境（ノートブック側で定義済み）
#   - PPO/SAC… : Stable-Baselines3 の各アルゴリズムクラス（PPO, SAC, TD3, DDPG, A2C）
#   - THETA_INIT, DT, L: 環境に関連するパラメータ（ノートブックで定義済み想定）
#   - ./logs/   : 学習時に保存しているフォルダ
# =============================================================================

# -----------------------------------------------------------------------------
# 1) 最新のログフォルダを見つけ、best_model.zip を探す
#    - 「どの実験結果を可視化するか」を人手ではなく自動で選びたいので、
#      更新日時が最も新しい logs サブフォルダを対象にする。
# -----------------------------------------------------------------------------
log_base = "./logs"
log_dirs = [d for d in glob.glob(os.path.join(log_base, "*")) if os.path.isdir(d)]
if not log_dirs:
    raise FileNotFoundError("logs フォルダにサブディレクトリがありません。学習結果を保存してください。")

latest_log_dir = max(log_dirs, key=os.path.getmtime)

# best_model.zip がある想定。なければフォールバックとして、フォルダ内の最新 .zip を採用する。
best_model_path = os.path.join(latest_log_dir, "best_model.zip")
if not os.path.exists(best_model_path):
    # フォールバック: 同ディレクトリにある .zip をすべて列挙し、最も新しいものを選ぶ
    zips = sorted(glob.glob(os.path.join(latest_log_dir, "*.zip")), key=os.path.getmtime)
    if not zips:
        raise FileNotFoundError(
            f"{best_model_path} が存在せず、{latest_log_dir} に .zip ファイルも見つかりません。"
        )
    best_model_path = zips[-1]

print(f"✅ 最新のモデルファイルを使用: {best_model_path}")
# 例: best_model_path = "logs/2025_11_19_22_23_03/best_model.zip"

# -----------------------------------------------------------------------------
# 便利関数: ラップを外して base 環境を得る
#   - VecEnv, Monitor, TimeLimit など SB3 由来のラッパをすべて剥がし、
#     本体の環境インスタンスにアクセスできるようにする。
#   - 内部状態 (theta, hand_pos など) を直接参照するために必要。
# -----------------------------------------------------------------------------
def unwrap_env(e):
    """
    ラッパー (VecEnv, Monitor, DummyVecEnv など) があれば中身の base 環境を返す。
    再帰的に unwrap するので、どんなラップ構造でも「一番内側の環境」に到達できる。
    """
    try:
        # VecEnv 系 (DummyVecEnv, SubprocVecEnv) は .envs に環境リストを持っている
        if hasattr(e, "envs") and isinstance(getattr(e, "envs"), (list, tuple)) and len(e.envs) > 0:
            return unwrap_env(e.envs[0])
    except Exception:
        pass

    try:
        # Monitor / TimeLimit 等は .env に一つ内側を保持していることが多い
        if hasattr(e, "env") and e.env is not None:
            return unwrap_env(e.env)
    except Exception:
        pass

    # これ以上 unwrap できなければ、それが base 環境とみなす
    return e

# -----------------------------------------------------------------------------
# 2) best_model をロード（どのアルゴリズムか分からない場合を想定）
#   - 学習時にどの SB3 アルゴリズムを使ったかを覚えていなくても、
#     代表的なクラスを順番に試し、ロードに成功したものを採用する。
# -----------------------------------------------------------------------------
loaded_model = None

# 試行順: PPO, SAC, TD3, DDPG, A2C
for alg in (PPO, SAC, TD3, DDPG, A2C):
    try:
        loaded_model = alg.load(best_model_path)
        print(f"✅ モデルを {alg.__name__} 形式でロードしました。")
        break
    except Exception:
        # 読み込み失敗は無視して次のアルゴリズムを試す
        continue

if loaded_model is None:
    raise RuntimeError("モデルのロードに失敗しました。対応する SB3 アルゴリズムを確認してください。")

best_model = loaded_model  # 以降で使う変数名を統一

# -----------------------------------------------------------------------------
# 3) env のリセットと初期観測の取得
#   - ノートブック側で env が既に定義されている前提。
#   - Gym / Gymnasium で reset の返り値が変わっているので両対応する。
# -----------------------------------------------------------------------------
try:
    obs_reset = env.reset()
    # Gymnasium: (obs, info), 旧 Gym: obs のみ
    if isinstance(obs_reset, tuple) and len(obs_reset) >= 1:
        obs = obs_reset[0]
    else:
        obs = obs_reset
except Exception as e:
    raise RuntimeError(
        "env.reset() に失敗しました。env が正しく定義・初期化されているか確認してください。"
    ) from e

# 内部状態を読むため、ラップを剥がした base_env を取得
base_env = unwrap_env(env)

# -----------------------------------------------------------------------------
# 4) ログ用のリストを初期化（2 関節アーム＋手先ノルム＋トルク）
#   - time_log を軸に、各物理量を列ごとのリストとして蓄積していく。
# -----------------------------------------------------------------------------
time_log = []
hand_x, hand_y = [], []

# 関節ごとのログ（joint1 / joint2）
theta1_log, theta2_log = [], []
theta1_vel_log, theta2_vel_log = [], []
theta1_acc_log, theta2_acc_log = [], []
theta1_jerk_log, theta2_jerk_log = [], []

# トルクログ [N·m]
torque1_log, torque2_log = [], []
action_same_log = []  # 2 関節トルクがほぼ同じかどうか

# 手先ノルムログ（速度, 加速度, 躍度）
hand_speed_log = []
hand_acc_norm_log = []
hand_jerk_norm_log = []

# -----------------------------------------------------------------------------
# 5) 初期状態を取得して、時刻 t=0 の行をログに追加する
#   - reset 直後の状態も後から解析したいので、最初のステップから記録する。
#   - base_env が状態を持っていない場合に備え、フォールバックを用意。
# -----------------------------------------------------------------------------
t = 0.0
time_log.append(t)

try:
    base = base_env  # alias

    # theta 系（存在しなければ THETA_INIT を使う）
    init_theta = np.asarray(
        getattr(base, "theta", np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)]))
    ).reshape(-1)[:2]
    init_theta_vel = np.asarray(
        getattr(base, "theta_vel", np.zeros(2))
    ).reshape(-1)[:2]
    init_theta_acc = np.asarray(
        getattr(base, "theta_acc", np.zeros(2))
    ).reshape(-1)[:2]
    init_theta_jerk = np.asarray(
        getattr(base, "theta_jerk", np.zeros(2))
    ).reshape(-1)[:2]

    # 手先位置: hand_pos があればそれを優先。
    if hasattr(base, "hand_pos"):
        init_hand = np.asarray(getattr(base, "hand_pos")).astype(float)
    else:
        # なければ forward_kinematics か、リンク長 L から単純な FK を行う
        if hasattr(base, "forward_kinematics"):
            init_hand = base.forward_kinematics(init_theta)
        else:
            L_val = float(getattr(base, "l", getattr(base, "l1", 1.0) + getattr(base, "l2", 0.0)))
            init_hand = L_val * np.array([np.cos(init_theta[0]), np.sin(init_theta[0])])
except Exception:
    # base_env に期待した属性がない場合のフォールバック
    init_theta = np.array([np.radians(THETA_INIT), np.radians(THETA_INIT)])
    init_theta_vel = np.zeros(2)
    init_theta_acc = np.zeros(2)
    init_theta_jerk = np.zeros(2)
    try:
        init_hand = np.asarray(getattr(base_env, "hand_pos", np.array([0.0, 0.0])))
    except Exception:
        init_hand = np.array([0.0, 0.0])

# --- 初期値をログに追加（角度は deg、位置は SI 単位） ---
hand_x.append(float(init_hand[0]))
hand_y.append(float(init_hand[1]))
theta1_log.append(np.degrees(init_theta[0]))
theta2_log.append(np.degrees(init_theta[1]))
theta1_vel_log.append(np.degrees(init_theta_vel[0]))
theta2_vel_log.append(np.degrees(init_theta_vel[1]))
theta1_acc_log.append(np.degrees(init_theta_acc[0]))
theta2_acc_log.append(np.degrees(init_theta_acc[1]))
theta1_jerk_log.append(np.degrees(init_theta_jerk[0]))
theta2_jerk_log.append(np.degrees(init_theta_jerk[1]))

# 手先ノルム初期値（速度, 加速度, 躍度）
init_hand_vel = np.asarray(getattr(base_env, "hand_vel", np.zeros(2))).reshape(-1)[:2]
init_hand_acc = np.asarray(getattr(base_env, "hand_acc", np.zeros(2))).reshape(-1)[:2]
init_hand_jerk = np.asarray(getattr(base_env, "hand_jerk", np.zeros(2))).reshape(-1)[:2]

hand_speed_log.append(float(np.linalg.norm(init_hand_vel)))
hand_acc_norm_log.append(float(np.linalg.norm(init_hand_acc)))
hand_jerk_norm_log.append(float(np.linalg.norm(init_hand_jerk)))

# t=0 は「まだ制御していない」のでトルクは NaN にしておく
torque1_log.append(np.nan)
torque2_log.append(np.nan)
action_same_log.append(False)

# -----------------------------------------------------------------------------
# 6) シミュレーション実行ループ（best_model を用いて行動を生成）
#   - Gym / Gymnasium 両対応の step 返り値をハンドリング。
#   - 各ステップで手先・関節・トルクをログに追加。
# -----------------------------------------------------------------------------
MAX_STEPS_RUN = 2000  # 可視化用途なので、必要に応じてここを増減させる

for step_i in range(MAX_STEPS_RUN):
    # 6-1. モデルから関節トルク [N·m] を取得
    action, _ = best_model.predict(obs, deterministic=True)
    action = np.asarray(action).reshape(-1)
    if action.shape[0] != 2:
        raise RuntimeError(
            f"predict returned action with wrong shape {action.shape}. expected (2,) for 2-joint environment."
        )

    # 6-2. env.step 実行（Gym / Gymnasium の返り値に対応）
    next_return = env.step(action)

    if isinstance(next_return, tuple) and len(next_return) == 5:
        # Gymnasium: (obs, reward, terminated, truncated, info)
        obs, reward, terminated, truncated, info = next_return
        done = bool(terminated or truncated)
    elif isinstance(next_return, tuple) and len(next_return) == 4:
        # 旧 Gym: (obs, reward, done, info)
        obs, reward, done, info = next_return
        terminated = bool(done)
        truncated = False
    else:
        raise RuntimeError("env.step() の返り値の形式が不明です。")

    # 6-3. 時刻を進める
    dt = getattr(base_env, "dt", None)
    if dt is None:
        # env が dt を持っていない場合は DT を参照、それもなければ 0.01 [s]
        try:
            dt = float(DT)
        except Exception:
            dt = 0.01
    t += dt
    time_log.append(t)

    # 6-4. info を優先して状態を取得（info にその時点の値を詰めている設計のため）
    if info is not None:
        # 手先位置
        if "hand_pos" in info:
            hand_pos = np.asarray(info["hand_pos"]).reshape(-1)[:2]
        else:
            # hand_pos がない場合は theta から順運動学で推定
            if hasattr(base_env, "forward_kinematics"):
                try:
                    theta_info = np.asarray(
                        info.get("theta", getattr(base_env, "theta", init_theta))
                    ).reshape(-1)[:2]
                    hand_pos = base_env.forward_kinematics(theta_info)
                except Exception:
                    hand_pos = np.array([np.nan, np.nan])
            else:
                # forward_kinematics がなくても最低限の位置を推定できるようにしておく
                try:
                    L_val = float(
                        getattr(base_env, "l", getattr(base_env, "l1", 1.0) + getattr(base_env, "l2", 0.0))
                    )
                    theta0 = float(
                        np.asarray(info.get("theta", getattr(base_env, "theta", init_theta)))[0]
                    )
                    hand_pos = L_val * np.array([np.cos(theta0), np.sin(theta0)])
                except Exception:
                    hand_pos = np.array([np.nan, np.nan])

        # 関節角・速度・加速度・躍度
        theta_arr = np.asarray(
            info.get("theta", getattr(base_env, "theta", init_theta))
        ).reshape(-1)[:2]
        theta_vel_arr = np.asarray(
            info.get("theta_vel", getattr(base_env, "theta_vel", init_theta_vel))
        ).reshape(-1)[:2]
        theta_acc_arr = np.asarray(
            info.get("theta_acc", getattr(base_env, "theta_acc", init_theta_acc))
        ).reshape(-1)[:2]
        theta_jerk_arr = np.asarray(
            info.get("theta_jerk", getattr(base_env, "theta_jerk", init_theta_jerk))
        ).reshape(-1)[:2]

        # 手先速度・加速度・躍度
        hand_vel = np.asarray(
            info.get("hand_vel", getattr(base_env, "hand_vel", np.zeros(2)))
        ).reshape(-1)[:2]
        hand_acc = np.asarray(
            info.get("hand_acc", getattr(base_env, "hand_acc", np.zeros(2)))
        ).reshape(-1)[:2]
        hand_jerk = np.asarray(
            info.get("hand_jerk", getattr(base_env, "hand_jerk", np.zeros(2)))
        ).reshape(-1)[:2]
    else:
        # info が空の場合は base_env の属性を直接読む
        try:
            theta_arr = np.asarray(getattr(base_env, "theta", init_theta)).reshape(-1)[:2]
            if hasattr(base_env, "forward_kinematics"):
                hand_pos = base_env.forward_kinematics(theta_arr)
            else:
                L_val = float(
                    getattr(base_env, "l", getattr(base_env, "l1", 1.0) + getattr(base_env, "l2", 0.0))
                )
                hand_pos = L_val * np.array([np.cos(theta_arr[0]), np.sin(theta_arr[0])])

            theta_vel_arr = np.asarray(getattr(base_env, "theta_vel", init_theta_vel)).reshape(-1)[:2]
            theta_acc_arr = np.asarray(getattr(base_env, "theta_acc", init_theta_acc)).reshape(-1)[:2]
            theta_jerk_arr = np.asarray(getattr(base_env, "theta_jerk", init_theta_jerk)).reshape(-1)[:2]

            hand_vel = np.asarray(getattr(base_env, "hand_vel", np.zeros(2))).reshape(-1)[:2]
            hand_acc = np.asarray(getattr(base_env, "hand_acc", np.zeros(2))).reshape(-1)[:2]
            hand_jerk = np.asarray(getattr(base_env, "hand_jerk", np.zeros(2))).reshape(-1)[:2]
        except Exception:
            # base_env に必要な情報がない場合の最後のフォールバック
            theta_arr = init_theta
            theta_vel_arr = init_theta_vel
            theta_acc_arr = init_theta_acc
            theta_jerk_arr = init_theta_jerk
            hand_pos = np.array([np.nan, np.nan])
            hand_vel = np.zeros(2)
            hand_acc = np.zeros(2)
            hand_jerk = np.zeros(2)

    # 6-5. ログに追加（位置・角度・トルク・ノルム）
    hand_x.append(float(hand_pos[0]))
    hand_y.append(float(hand_pos[1]))
    theta1_log.append(np.degrees(theta_arr[0]))
    theta2_log.append(np.degrees(theta_arr[1]))
    theta1_vel_log.append(np.degrees(theta_vel_arr[0]))
    theta2_vel_log.append(np.degrees(theta_vel_arr[1]))
    theta1_acc_log.append(np.degrees(theta_acc_arr[0]))
    theta2_acc_log.append(np.degrees(theta_acc_arr[1]))
    theta1_jerk_log.append(np.degrees(theta_jerk_arr[0]))
    theta2_jerk_log.append(np.degrees(theta_jerk_arr[1]))

    # トルク記録 [N·m]（info["action"] があればそれを「実際にかけた値」として優先）
    if info is not None and "action" in info:
        torque_vec = np.asarray(info["action"]).reshape(-1)
    else:
        torque_vec = action  # env.step に渡したもの

    torque1_log.append(float(torque_vec[0]))
    torque2_log.append(float(torque_vec[1]))
    action_same_log.append(bool(np.allclose(torque_vec[0], torque_vec[1], atol=1e-6)))

    # 手先ノルム（SI 単位）を計算してログ
    hand_speed_log.append(float(np.linalg.norm(hand_vel)))
    hand_acc_norm_log.append(float(np.linalg.norm(hand_acc)))
    hand_jerk_norm_log.append(float(np.linalg.norm(hand_jerk)))

    # 終端条件（terminated / truncated）または異常な長さで break
    if bool(terminated) or bool(truncated) or (len(time_log) > 100000):
        break

# -----------------------------------------------------------------------------
# 7) DataFrame 作成 & CSV 保存（2関節・手先ノルム・トルク列）
#   - 解析用にすべてのログを 1 つの CSV にまとめる。
# -----------------------------------------------------------------------------
df = pd.DataFrame({
    "Time": time_log,
    "HandX": hand_x,
    "HandY": hand_y,
    "HandSpeed (m/s)": hand_speed_log,
    "HandAcc (m/s^2)": hand_acc_norm_log,
    "HandJerk (m/s^3)": hand_jerk_norm_log,
    "Theta1 (deg)": theta1_log,
    "Theta2 (deg)": theta2_log,
    "Theta1_vel (deg/s)": theta1_vel_log,
    "Theta2_vel (deg/s)": theta2_vel_log,
    "Theta1_acc (deg/s^2)": theta1_acc_log,
    "Theta2_acc (deg/s^2)": theta2_acc_log,
    "Theta1_jer (deg/s^3)": theta1_jerk_log,
    "Theta2_jer (deg/s^3)": theta2_jerk_log,
    "Torque1 (N·m)": torque1_log,
    "Torque2 (N·m)": torque2_log,
    "Action_same": action_same_log
})

output_dir = latest_log_dir
os.makedirs(output_dir, exist_ok=True)
csv_path = os.path.join(output_dir, "best_result_2joint_with_initial_and_handnorms.csv")
df.to_csv(csv_path, index=False)
print(f"✅ CSV 保存: {csv_path}")

# -----------------------------------------------------------------------------
# 8) プロット群の生成: 手先軌跡、関節ダイナミクス、手先ノルム、トルク
#   - 図の出力パスは output_dir にまとめて保存する。
#   - 研究ノートにそのまま貼れるクオリティを想定している。
# -----------------------------------------------------------------------------

# ===== 8-1. 手先軌道 + アーム姿勢を重ねる =====
fig1, ax1 = plt.subplots(figsize=(6, 6))

ax1.plot(df["HandX"], df["HandY"], linestyle="-", linewidth=1.5, label="end-effector trajectory")
ax1.scatter(df["HandX"].iloc[0], df["HandY"].iloc[0], color="red", label="start", zorder=5)
ax1.scatter(df["HandX"].iloc[-1], df["HandY"].iloc[-1], color="green", label="end", zorder=5)

# リンク長（env から取得できなければ L を半分ずつ使う）
try:
    l1 = float(getattr(env, "l1", (L * 0.5)))
    l2 = float(getattr(env, "l2", (L * 0.5)))
except Exception:
    l1 = L * 0.5
    l2 = L * 0.5

theta1_rad = np.radians(df["Theta1 (deg)"].values)
theta2_rad = np.radians(df["Theta2 (deg)"].values)

x1 = l1 * np.cos(theta1_rad)
y1 = l1 * np.sin(theta1_rad)
x2 = x1 + l2 * np.cos(theta1_rad + theta2_rad)
y2 = y1 + l2 * np.sin(theta1_rad + theta2_rad)

# FK 計算結果と HandX/Y の乖離をチェック（環境バグの早期発見用）
try:
    mismatch = np.nanmax(
        np.abs(
            np.column_stack([df["HandX"].values, df["HandY"].values]) -
            np.column_stack([x2, y2])
        )
    )
    if mismatch > 1e-6:
        print(f"[WARN] forward kinematics and logged hand pos mismatch (max diff={mismatch:.6e})")
except Exception:
    pass

# アーム姿勢を時系列に沿って薄いグレーで重ね描画
N_POSES = min(15, len(df))
indices = np.linspace(0, len(df) - 1, N_POSES).astype(int)

for idx in indices:
    xs = [0.0, x1[idx], x2[idx]]
    ys = [0.0, y1[idx], y2[idx]]
    ax1.plot(xs, ys, linewidth=1, color=(0.5, 0.5, 0.5, 0.25), solid_capstyle='round')
    ax1.scatter(
        [xs[1], xs[2]],
        [ys[1], ys[2]],
        s=40,
        edgecolors='k',
        facecolors=(0.9, 0.9, 0.9, 0.6),
        zorder=4
    )

# 強調: start / end 姿勢
i0 = 0
xs0 = [0.0, x1[i0], x2[i0]]
ys0 = [0.0, y1[i0], y2[i0]]
ax1.plot(xs0, ys0, linewidth=2, color='tab:red', solid_capstyle='round', zorder=6)
ax1.scatter([xs0[1], xs0[2]], [ys0[1], ys0[2]], s=80, color='tab:red', edgecolors='k', zorder=7)
ax1.scatter(0.0, 0.0, s=100, color='black', zorder=8)

ie = len(df) - 1
xse = [0.0, x1[ie], x2[ie]]
yse = [0.0, y1[ie], y2[ie]]
ax1.plot(xse, yse, linewidth=2, color='tab:green', solid_capstyle='round', zorder=6)
ax1.scatter([xse[1], xse[2]], [yse[1], yse[2]], s=100, color='tab:green', edgecolors='k', zorder=7)

ax1.set_xlabel("X [m]")
ax1.set_ylabel("Y [m]")
ax1.grid(True)
ax1.set_aspect('equal', adjustable='box')

reach = l1 + l2
ax1.set_xlim(-reach - 0.1 * reach, reach + 0.1 * reach)
ax1.set_ylim(-0.1 * reach, reach + 0.6 * reach)

base_filename = os.path.join(output_dir, "best_result_2joint_with_handnorms")
plt.tight_layout()
plt.savefig(f"{base_filename}_trajectory_with_arms.png", dpi=300)
plt.show()

# ===== 8-2. 関節動的変数（角度・速度・加速度・躍度） =====
fig2, axes = plt.subplots(4, 1, figsize=(8, 12), sharex=True)

axes[0].plot(df["Time"], df["Theta1 (deg)"], label="Theta1")
axes[0].plot(df["Time"], df["Theta2 (deg)"], linestyle="--", label="Theta2")
axes[0].set_ylabel("Theta (deg)")
axes[0].legend()
axes[0].grid()
axes[0].set_ylim(0, 180)

axes[1].plot(df["Time"], df["Theta1_vel (deg/s)"], label="Vel1")
axes[1].plot(df["Time"], df["Theta2_vel (deg/s)"], linestyle="--", label="Vel2")
axes[1].set_ylabel("Velocity (deg/s)")
axes[1].legend()
axes[1].grid()

axes[2].plot(df["Time"], df["Theta1_acc (deg/s^2)"], label="Acc1")
axes[2].plot(df["Time"], df["Theta2_acc (deg/s^2)"], linestyle="--", label="Acc2")
axes[2].set_ylabel("Acceleration (deg/s^2)")
axes[2].legend()
axes[2].grid()

axes[3].plot(df["Time"], df["Theta1_jer (deg/s^3)"], label="Jerk1")
axes[3].plot(df["Time"], df["Theta2_jer (deg/s^3)"], linestyle="--", label="Jerk2")
axes[3].set_ylabel("Jerk (deg/s^3)")
axes[3].set_xlabel("Time (s)")
axes[3].legend()
axes[3].grid()

png_dynamics = os.path.join(output_dir, "best_result_2joint_joint_dynamics.png")
plt.tight_layout()
plt.savefig(png_dynamics, dpi=300)
plt.show()
print(f"✅ 関節動的変数プロット保存: {png_dynamics}")

# ===== 8-3. 手先ノルム（速度・加速度・躍度） =====
fig3, ax3 = plt.subplots(3, 1, figsize=(8, 9), sharex=True)

ax3[0].plot(df["Time"], df["HandSpeed (m/s)"], label="||v||")
ax3[0].set_ylabel("Speed (m/s)")
ax3[0].legend()
ax3[0].grid()

ax3[1].plot(df["Time"], df["HandAcc (m/s^2)"], label="||a||")
ax3[1].set_ylabel("Acc (m/s^2)")
ax3[1].legend()
ax3[1].grid()

ax3[2].plot(df["Time"], df["HandJerk (m/s^3)"], label="||jerk||")
ax3[2].set_ylabel("Jerk (m/s^3)")
ax3[2].set_xlabel("Time (s)")
ax3[2].legend()
ax3[2].grid()

png_handnorms = os.path.join(output_dir, "best_result_2joint_hand_norms.png")
plt.tight_layout()
plt.savefig(png_handnorms, dpi=300)
plt.show()
print(f"✅ 手先ノルムプロット保存: {png_handnorms}")

# ===== 8-4. トルクの遷移と action_same フラグ =====
fig4, ax4 = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

ax4[0].plot(df["Time"], df["Torque1 (N·m)"], label="Torque1")
ax4[0].plot(df["Time"], df["Torque2 (N·m)"], linestyle="--", label="Torque2")
ax4[0].set_ylabel("Torque (N·m)")
ax4[0].legend()
ax4[0].grid()

ax4[1].plot(df["Time"], df["Action_same"].astype(int), label="Action_same")
ax4[1].set_ylabel("Action_same")
ax4[1].set_xlabel("Time (s)")
ax4[1].grid()

png_torques = os.path.join(output_dir, "best_result_2joint_torques.png")
plt.tight_layout()
plt.savefig(png_torques, dpi=300)
plt.show()
print(f"✅ トルクプロット保存: {png_torques}")

# ===== 完了メッセージ =====
print("=== 実行完了: 2関節対応の評価ログ & プロットを保存しました ===")
print(f"出力フォルダ: {output_dir}")


## エピソードごとの様々な報酬の推移などの多指標をプロット

In [ ]:
# =============================================================================
# episode_full_metrics.csv から学習ログをまとめて可視化するスクリプト
# =============================================================================
# 役割:
#   - ./logs 以下の「最新ログフォルダ」から episode_full_metrics.csv を自動検出
#   - 学習エピソードごとの各種指標（報酬、距離・躍度コスト、時間ボーナス、jerk_sum など）を読み込む
#   - 指標をグルーピングしてまとめプロット＋個別プロットを自動生成し、PNG 画像として保存する
#   - すべてのプロットについて「生データ版」と「移動平均版（rolling mean）」を出力する
#
# 設計方針:
#   - 「あとから列が増えたり減ったりしても壊れない」ことを重視し、
#       - 列が存在したものだけプロット
#       - グループ定義も存在チェックの上で描画
#   - X 軸は基本的に episode 番号だが、episode 列がなくてもインデックスで代用できる。
#   - 移動平均窓幅 MOVING_AVG_WINDOW を冒頭でまとめて設定し、ノートブック上で調整しやすくする。
#
# 関連コンポーネント:
#   - ./logs/****/episode_full_metrics.csv : 学習ループから出力されるメトリクス CSV
#   - label_unit_map : CSV 列名 → プロット用のラベル・単位への変換マップ
#   - groups         : まとめて重ねて見たい指標グループの定義
# =============================================================================

plt.rcParams["font.size"] = 12  # スライド・論文でも見やすいフォントサイズ

# -----------------------------------------------------------------------------
# 設定: 移動平均窓幅
#   - 学習がノイジーなときにトレンドを見やすくするためのパラメータ。
#   - エピソード数に応じて 10, 50, 100 などに変更して使う。
# -----------------------------------------------------------------------------
MOVING_AVG_WINDOW = 10

# -----------------------------------------------------------------------------
# 1) 最新ログフォルダの自動検出
#   - ./logs 配下のサブディレクトリを列挙し、更新日時が最も新しいものを使用する。
# -----------------------------------------------------------------------------
logs_dir = "./logs"
log_folders = [
    os.path.join(logs_dir, d)
    for d in os.listdir(logs_dir)
    if os.path.isdir(os.path.join(logs_dir, d))
]
if not log_folders:
    raise FileNotFoundError("`./logs` 内にフォルダが見つかりません。学習ログがあるディレクトリを確認してください。")

latest_log_folder = max(log_folders, key=os.path.getmtime)
print(f"[INFO] 最新ログフォルダ: {latest_log_folder}")

# -----------------------------------------------------------------------------
# 2) episode_full_metrics.csv のパス決定
#   - デフォルト名が見つからない場合は、名前が少し違っていても使えるように近い名前を探索する。
# -----------------------------------------------------------------------------
csv_filename = "episode_full_metrics.csv"
csv_path = os.path.join(latest_log_folder, csv_filename)

if not os.path.exists(csv_path):
    # ファイル名が微妙に違う場合に備えて、「episode」「full」を含む CSV を候補として探す
    candidates = glob.glob(os.path.join(latest_log_folder, "*episode*.csv")) \
                 + glob.glob(os.path.join(latest_log_folder, "*full*.csv"))
    if candidates:
        csv_path = candidates[-1]
        print(f"[WARN] {csv_filename} が見つかりませんでした。代わりに候補を使用します: {csv_path}")
    else:
        raise FileNotFoundError(f"{csv_filename} が見つかりません: {latest_log_folder}")

# -----------------------------------------------------------------------------
# 3) CSV 読み込み
#   - 通常はヘッダ 1 行目として読み込む。
#   - エクスポートの仕方によっては 1 行目がコメントになっていることもあるので、
#     エラー時は skiprows=1 で再トライする。
# -----------------------------------------------------------------------------
try:
    df = pd.read_csv(csv_path)
    print(f"[INFO] CSV を読み込みました: {csv_path}")
except Exception as e:
    print("[WARN] 直接読み込みに失敗しました。skiprows=1 を試します。エラー:", e)
    df = pd.read_csv(csv_path, skiprows=1)
    print(f"[INFO] CSV を読み込みました（skiprows=1）: {csv_path}")

# -----------------------------------------------------------------------------
# 4) 列名 → ラベル・単位のマッピング
#   - プロットの軸ラベルに人間が読める形の名称と単位を付けたいので、
#     生の列名からラベル・単位を引ける辞書を用意しておく。
# -----------------------------------------------------------------------------
label_unit_map = {
    "episode":                ("Episode", ""),
    "episode_length":         ("Episode length", "steps"),
    "total_reward":           ("Total reward", "arb."),  # 報酬は設計依存なので単位は任意
    "sum_reward_dist_step":   ("Sum distance shaping reward", "arb."),
    "sum_reward_jerk_step":   ("Sum jerk shaping reward", "arb."),
    "sum_reward_time_step":   ("Sum time shaping reward", "arb."),
    "sum_terminal_jerk_penalty": ("Terminal jerk penalty", "arb."),
    "sum_terminal_vel_penalty":  ("Terminal velocity penalty", "arb."),
    "sum_time_bonus":         ("Terminal time bonus", "arb."),
    "jerk_sum":               ("Jerk sum", "rad^2/s^5"),
    "running_mean_total_reward_100": ("Running mean total reward (100)", "arb."),
    "episode_wall_time":      ("Episode wall time", "s"),
    "sigma_T":                ("sigma_T", ""),
    "shaping_scale":          ("shaping_scale", "")
    # 必要になったら loss 系や success 系もここに追加する
}

# -----------------------------------------------------------------------------
# 5) 利用可能な列名を表示（デバッグ・確認用）
# -----------------------------------------------------------------------------
print("[INFO] CSV の列:")
for c in df.columns:
    print("  ", c)

# -----------------------------------------------------------------------------
# 6) X 軸に使う episode 番号を決定
#   - 本来は episode 列を使うが、ない場合は単純に 1..N をエピソード番号として扱う。
# -----------------------------------------------------------------------------
if "episode" in df.columns:
    episodes = df["episode"].values
else:
    episodes = np.arange(1, len(df) + 1)
    print("[WARN] 'episode' 列が見つかりません。インデックスをエピソード番号として使用します。")

# -----------------------------------------------------------------------------
# 7) グループ定義：同じ図にまとめて重ねたい列を定義
#   - 報酬成分だけをまとめる、ステップ系と終端系を分ける、など
#     「比較して見たいもの」を 1 枚の図に載せる。
# -----------------------------------------------------------------------------
groups = {
    "reward_components": [
        "total_reward",
        "sum_terminal_jerk_penalty",
        "sum_terminal_vel_penalty",
        "sum_time_bonus",
        "sum_reward_dist_step",
        "sum_reward_jerk_step",
        "sum_reward_time_step",
    ],
    "reward_goal_components": [
        "sum_terminal_jerk_penalty",
        "sum_terminal_vel_penalty",
        "sum_time_bonus"
    ],
    "reward__step_components": [
        "sum_reward_dist_step",
        "sum_reward_jerk_step",
        "sum_reward_time_step",
    ],
    "dynamics": [
        "jerk_sum",
        "episode_length",
        "episode_wall_time"
    ]
    # 例えば学習安定性を見たいときは learning_metrics / running_stats を追加定義する
}

# -----------------------------------------------------------------------------
# 8) 保存ディレクトリの準備
#   - 出力先: 最新ログフォルダ/plots 以下にすべての PNG をまとめる。
# -----------------------------------------------------------------------------
out_dir = latest_log_folder
plots_dir = os.path.join(out_dir, "plots")
os.makedirs(plots_dir, exist_ok=True)

# -----------------------------------------------------------------------------
# 9) 描画ヘルパー関数
#   - safe_get_col: 列がなければ None を返す
#   - plot_series: 1 or 多系列の折れ線グラフを保存
# -----------------------------------------------------------------------------
def safe_get_col(df, col):
    """DataFrame から列を安全に取得する。存在しなければ None を返す。"""
    return df[col].values if col in df.columns else None

def plot_series(x, y, xlabel, ylabel, savepath, legend=None, ylim=None):
    """
    単一系列または複数系列を描画して保存するヘルパー。

    Parameters
    ----------
    x : array-like
        X 軸（エピソード番号など）
    y : array-like or dict[str, array-like]
        Y 軸データ。dict の場合は key を凡例名として複数系列を重ね描画。
    xlabel, ylabel : str
        軸ラベル。
    savepath : str
        保存先 PNG ファイルパス。
    legend : bool or None
        True のとき凡例を表示。None のときは表示しない。
    ylim : tuple or None
        Y 軸の範囲を指定したいときに使用。
    """
    if y is None:
        print(f"[SKIP] {savepath} : データが存在しないためスキップします。")
        return

    plt.figure(figsize=(10, 4.5))

    if isinstance(y, dict):
        # 複数系列
        for name, arr in y.items():
            plt.plot(x, arr, label=name)
    else:
        # 単一系列
        plt.plot(x, y)

    plt.xlabel(xlabel)
    plt.ylabel(ylabel)

    if legend:
        plt.legend(loc="best")

    plt.grid(True)

    if ylim is not None:
        plt.ylim(ylim)

    plt.tight_layout()
    plt.savefig(savepath, dpi=300)
    plt.close()
    print(f"[SAVED] {savepath}")

# -----------------------------------------------------------------------------
# 10) 移動平均を計算するヘルパー関数
#   - NaN を許容しつつ単純移動平均を返す（pandas の rolling を活用）。
# -----------------------------------------------------------------------------
def moving_average(arr, window):
    """NaN を許容しつつ単純移動平均を返す。pandas.Series.rolling を利用。"""
    if arr is None:
        return None
    return pd.Series(arr).rolling(window=window, min_periods=1).mean().to_numpy()

# -----------------------------------------------------------------------------
# 11) グループごとのまとめプロット
#   - 各グループについて、
#       (1) 元データ版の重ね描画
#       (2) 移動平均版の重ね描画
#     の 2 種類を出力する。
# -----------------------------------------------------------------------------
for gname, cols in groups.items():
    # 実際に CSV に存在する列だけを対象にする
    available = [c for c in cols if c in df.columns]
    if not available:
        print(f"[SKIP] グループ '{gname}' にプロット可能な列がありません。")
        continue

    # --- 元データでのプロット ---
    series_dict = {}
    for c in available:
        arr = df[c].values
        label, unit = label_unit_map.get(c, (c, ""))
        label_full = f"{label} [{unit}]" if unit else label
        series_dict[label_full] = arr

    savepath = os.path.join(plots_dir, f"group_{gname}.png")
    plot_series(
        episodes,
        series_dict,
        xlabel="Episode",
        ylabel="Value",
        savepath=savepath,
        legend=True
    )

    # --- 移動平均版のプロット ---
    ma_series_dict = {}
    for c in available:
        arr = df[c].values
        arr_ma = moving_average(arr, MOVING_AVG_WINDOW)
        label, unit = label_unit_map.get(c, (c, ""))
        label_full = f"{label} [{unit}]" if unit else label
        ma_series_dict[label_full] = arr_ma

    savepath_ma = os.path.join(plots_dir, f"group_{gname}_ma_w{MOVING_AVG_WINDOW}.png")
    plot_series(
        episodes,
        ma_series_dict,
        xlabel="Episode",
        ylabel=f"Mean Value (window={MOVING_AVG_WINDOW})",
        savepath=savepath_ma,
        legend=True
    )

# -----------------------------------------------------------------------------
# 12) 各列ごとの個別プロット
#   - 全列に対して 1 枚ずつのプロットを作る。
#   - success 系は 0〜1 の比率なので ylim を 0〜1 に固定するなど、列に応じて調整。
# -----------------------------------------------------------------------------
for col in df.columns:
    if col == "episode":
        continue

    arr = safe_get_col(df, col)
    if arr is None:
        continue

    label, unit = label_unit_map.get(col, (col, ""))
    ylabel = f"{label} [{unit}]" if unit else label
    savepath = os.path.join(plots_dir, f"episode_{col}.png")

    # success / running_success_rate の場合は 0〜1 の比率
    ylim = None
    if col in ("success", "running_success_rate_100"):
        ylim = (-0.05, 1.05)

    # 元データ版
    plot_series(
        episodes,
        arr,
        xlabel="Episode",
        ylabel=ylabel,
        savepath=savepath,
        legend=False,
        ylim=ylim
    )

    # 移動平均版
    arr_ma = moving_average(arr, MOVING_AVG_WINDOW)
    savepath_ma = os.path.join(plots_dir, f"episode_{col}_ma_w{MOVING_AVG_WINDOW}.png")
    plot_series(
        episodes,
        arr_ma,
        xlabel="Episode",
        ylabel=f"{label} (moving avg)",
        savepath=savepath_ma,
        legend=False,
        ylim=ylim
    )

# -----------------------------------------------------------------------------
# 13) 全報酬成分を重ねた特別プロット
#   - reward_components グループの列だけを取り出し、
#     報酬成分のバランスや学習による変化を一望できる図を作る。
# -----------------------------------------------------------------------------
reward_cols = [c for c in groups["reward_components"] if c in df.columns]

if reward_cols:
    # 元データ版
    series = {}
    for c in reward_cols:
        label, unit = label_unit_map.get(c, (c, ""))
        series[f"{label} [{unit}]"] = df[c].values

    savepath = os.path.join(plots_dir, "all_reward_components_overlay.png")
    plot_series(
        episodes,
        series,
        xlabel="Episode",
        ylabel="Reward (arb.)",
        savepath=savepath,
        legend=True
    )

    # 移動平均版
    series_ma = {}
    for c in reward_cols:
        label, unit = label_unit_map.get(c, (c, ""))
        series_ma[f"{label} [{unit}]"] = moving_average(df[c].values, MOVING_AVG_WINDOW)

    savepath_ma = os.path.join(plots_dir, f"all_reward_components_overlay_ma_w{MOVING_AVG_WINDOW}.png")
    plot_series(
        episodes,
        series_ma,
        xlabel="Episode",
        ylabel="Reward (moving avg)",
        savepath=savepath_ma,
        legend=True
    )

print("[DONE] プロット作成完了。結果はフォルダに保存されています:", plots_dir)
